# Image Analysis videodis.co - jm V1.02

## Setup

### Import

In [10]:
import sys
#sys.path.append('../shared/')
#import videodisko as vd
# Imports
import numpy as np
import numpy.matlib
import numpy.random

from mpl_toolkits.mplot3d import axes3d 

from matplotlib import pyplot as plt

from scipy import ndimage
from scipy import misc
from scipy import fftpack
from scipy import optimize
import scipy.io
import skimage
import imageio
import colour as c
import math
import csv

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import pandas as pd

import cv2
import time

from datetime import datetime

import os
from fnmatch import fnmatch

### Luminance Encoding

In [11]:

## Linear to sRGB
def lin2srgb (x):
    result = (x <= 0.0031308) * np.multiply(x,12.92) + (x > 0.0031308) * np.subtract(np.multiply(1.055,np.power(x,(1/2.4))),0.055)
    return result


## sRGB to Linear 
def srgb2lin (x):
    result = (x <= 0.04045) * np.divide(x,12.92) + (x > 0.04045) * np.power(np.divide(np.add(x,0.055),1.055),2.4)
    return result

# SDR Formulas

## Input Luminance between 0-1
## Output Videosignal
def oetf709(x): 
    return np.subtract(np.multiply(np.power(x, 0.45), 1.099), 0.099)*(0.018<= x) + np.multiply(4.5, x) * (x <0.018)

def eotf1886(x, Lw = 100, Lb = 0): 
    gamma = 2.4
    a = np.power(np.subtract(np.power(Lw, np.divide(1,gamma)),np.power(Lb, np.divide(1, gamma))), gamma)
    b = np.divide(np.power(Lb, np.divide(1,gamma)) , np.subtract(np.power(Lw , np.divide(1,gamma)), np.power( Lb , np.divide(1, gamma))))
    return np.multiply(a , np.power( np.maximum(np.add(x, b), 0) , gamma))

def eotf1886_CRT(x, Lw = 100, Lb = 0, b = 0): 
    Vc = 0.35
    a1 = 2.6
    a2 = 3.0
    k = np.subtract(Lw,np.power(np.add(1,b),a1))
    return (x < Vc) * np.multiply(np.multiply(k,np.power(np.add(Vc,b),np.subtract(a1,a2))),np.power(np.add(x,b),a2)) + (Vc <= x) * np.multiply(k,np.power(np.add(x,b),a1))

## Luma Formulas for Rec2020 / Rec709 -> Fixed Addition
def luma709(img): 
    return np.multiply(img[:,:,0],0.2126) + np.multiply(img[:,:,1],0.7152) + np.multiply(img[:,:,2],0.0722)
def luma2020(img): 
    return np.multiply(img[:,:,0],0.2627) + np.multiply(img[:,:,1],0.6780) + np.multiply(img[:,:,2],0.0593)

## Formulas for PQ

# increasing rec.709 formulas 
def steroids709(x): 
    return np.subtract(np.multiply(np.power(np.multiply(59.5208 , x), 0.45), 1.099), 0.099)*(0.0003024< x) + np.multiply(267.84, x) * (x <= 0.0003024)

def steroids1886(x): 
    return np.multiply(100, np.power(x, 2.4))


## Transportfunctions

def pq_eotf(x): 
    ##m1
    m1 = np.divide(2610, 16384) ## 2^14 = 16384
    m2 = np.multiply(np.divide(2523, 4096), 128) ##2^12 = 4096 , 2^7 = 128 -> Jan fragen
    c1 = np.divide(3424, 4096)
    c2 = np.multiply(np.divide(2413, 4096), 32)
    c3 = np.multiply(np.divide(2392, 4096), 32)

    
    return np.multiply(np.power(np.divide(np.maximum( np.subtract( np.power(x , np.divide(1 , m2)), c1) , 0) , np.subtract(c2, np.multiply(c3 , np.power(x , np.divide(1 , m2))))) , np.divide(1, m1)), 10000)

## Inverse_eotf

def pq_eotf_inv(x): 
    m1 = np.divide(2610, 16384) ## 2^14 = 16384
    m2 = np.multiply(np.divide(2523, 4096), 128) ##2^12 = 4096 , 2^7 = 128 -> Jan fragen
    c1 = np.divide(3424, 4096)
    c2 = np.multiply(np.divide(2413, 4096), 32)
    c3 = np.multiply(np.divide(2392, 4096), 32)

    y = np.divide(x , 10000) ## Normalisierung auf 0.00 bis 1.0

    return np.power( np.divide(np.add(c1 , np.multiply(c2 , np.power(y , m1))), np.add(1 , np.multiply(c3 , np.power(y , m1)))), m2)

# RGB Input
def hlg_oetf(x): 
    a = 0.17883277
    b = np.subtract(1 , np.multiply(4 , a))
    c = np.subtract(0.5 , np.multiply(a , np.log(np.multiply(4 , a))))

    return np.where(x <= np.divide(1,12), np.sqrt(np.multiply(3,x )), np.add( np.multiply(a , np.log(np.subtract( np.multiply(12 , x ), b))) , c) )

def hlg_oetf_inv(x):
    a = 0.17883277
    b = np.subtract(1 , np.multiply(4 , a))
    c = np.subtract(0.5 , np.multiply(a , np.log(np.multiply(4 , a))))
    
    return np.where(x <= 0.5 , np.divide( np.power(x,2), 3), np.divide(np.add(np.exp(np.divide(np.subtract(x , c), a)), b) , 12) )

## Lw = Usergain, nominal peak Luminance of the display (achromatic)
def hlg_ootf(x, Lw): 
    r = x[: , : , 0]
    g = x[: , : , 1]
    b = x[: , : , 2]
    luma = np.add(np.multiply(0.2627 ,r ), np.multiply(0.678 , g), np.multiply(0.0593 , b))
    alpha = Lw
    gamma = np.add(1.2, np.multiply(0.42 , np.log10(np.divide(Lw,1000))))
    rd = np.multiply( alpha , np.power(luma , np.subtract(gamma , 1)), r)
    gd = np.multiply( alpha , np.power(luma , np.subtract(gamma , 1)), g)
    bd = np.multiply( alpha , np.power(luma , np.subtract(gamma , 1)), b)
    return np.stack((rd , gd , bd))

## Lw = Usergain, nominal peak Luminance of the display (achromatic)
def hlg_ootf_bw(x, Lw=1000): 
    alpha = Lw
    gamma = np.add(1.2, np.multiply(0.42 , np.log10(np.divide(Lw,1000))))
    return np.multiply(np.multiply( alpha , np.power(x , np.subtract(gamma , 1))),x)

def hlg_eotf(x, Lw = 1000, Lb = 0.005):
    gamma = np.add(1.2, np.multiply(0.42 , np.log10(np.divide(Lw,1000))))
    beta = np.sqrt(np.multiply(3 , np.power(np.divide(Lb , Lw), np.divide(1,gamma))))
    return hlg_ootf_bw(hlg_oetf_inv(np.maximum(0 , (np.add( np.multiply( np.subtract(1 , beta), x ), beta)))), Lw)


def slog3oetf(x): 
    # reflection add
    #vx = x*0.9

    return (x < 0.01125)* np.divide( np.add( np.multiply(x , np.divide(np.subtract(171.2102946929, 95), 0.01125)), 95), 1023) + (x >= 0.01125)* np.divide(np.add(420 , np.multiply( np.log10(np.divide(np.add(x,0.01), np.add(0.18, 0.01))), 261.5)), 1023)

def slog3eotf(x): 
    return (x >= np.divide(171.2102946929,1023)) * np.subtract( np.multiply(np.power(10 , np.divide( np.subtract(420 , np.multiply(x , 1023)) , 261.5)) , np.add(0.18 , 0.01)) , 0.01) + (x < np.divide(171.2102946929,1023)) * np.multiply(np.subtract(np.multiply(x , 1023), 95 ) , np.divide(0.01125 , np.subtract(171.2102946929 , 95)))

# These are the ARRI LogCv3 Formulas
def lin2logc(x):
    return np.multiply(x > 0.010591, 0.24719 * np.log10(5.555556 * x + 0.052272) + 0.385537) + np.multiply(x <= 0.010591, 5.367655 * x + 0.092809)
def logc2lin(x):
    return np.multiply(x > 0.149659, (np.power(10.0, (x - 0.385537) / 0.247189) - 0.052272) / 5.555556) + np.multiply(x <= 0.149659, (x - 0.092809) / 5.367650)

### LOG C4
def Encoding_LogC4(x):
    a = (np.power(2.0, 18.0) - 16.0) / 117.45
    b = (1023.0 - 95.0) / 1023.0
    c = 95.0 / 1023.0
    s = (7 * np.log(2) * np.power(2.0, 7 - 14 * c / b)) / (a * b)
    t = (np.power(2.0, 14.0 * (-c / b) + 6.0) - 64.0) / a
    return (x < t) * np.divide(np.subtract(x,t),s) + (x >= t) * np.add(np.multiply(np.divide( np.subtract(np.log2(np.add(np.multiply(a,x),64.0)),6.0),14.0),b),c)

def Decoding_LogC4(x):
    a = (np.power(2.0, 18.0) - 16.0) / 117.45
    b = (1023.0 - 95.0) / 1023.0
    c = 95.0 / 1023.0
    s = (7 * np.log(2) * np.power(2.0, 7 - 14 * c / b)) / (a * b)
    t = (np.power(2.0, 14.0 * (-c / b) + 6.0) - 64.0) / a
    return (x < 0.0) * np.add(np.multiply(x,s),t) + (x >= 0.0) * np.divide(np.subtract(np.power(2.0, np.add(np.multiply(14,np.divide(np.subtract(x,c),b)),6)),64.0),a)


def full2legal(x):
    return (219*(x)+16)*np.power(2,(10-8))

def legal2full(x):
    return (((x/np.power(2,(10-8)))-16)/219)

def lin2log2(x):
    return np.log2(x)

def log22lin(x):
    return np.power(2,x)

def Blenden_Differenz(num1, num2):
    new2 = num2 / num1
    return np.log2(new2)

def mse_calculation(imageA, imageB):
    # Expects Luma Images
	err = np.mean(np.square(np.subtract(imageA,imageB)))
	return err

In [28]:
test01 = c.read_image("/Volumes/jm_HDD/jm_MA_V1/CONTENT/XX_WATCHFOLDER_TIFF_DUBLICATION/13.1_TRANSFORMED_PQ_FR_HD/20231219_HdM_HDR_2023_VideoDisco_V2_Selection/TRANSFORMED_PQ_BT2020_FR_HD_HdM_HDR_2023_VideoDisco_LogC3_AWG3_TIFF_RGB_16_bit_190562/TRANSFORMED_PQ_BT2020_FR_HD_HdM_HDR_2023_VideoDisco_LogC3_AWG3_TIFF_RGB_16_bit_190562.tif")
test02 = c.read_image("/Volumes/jm_HDD/41_TIFF_EXTRACTION/31.1_PQ_UHD/01_ProRes_4444/13.1_TRANSFORMED_PQ_FR_HD/20231219_HdM_HDR_2023_VideoDisco_V2_Selection/TRANSFORMED_PQ_BT2020_FR_HD_HdM_HDR_2023_VideoDisco_LogC3_AWG3_TIFF_RGB_16_bit_190562/TRANSFORMED_PQ_BT2020_FR_HD_HdM_HDR_2023_VideoDisco_LogC3_AWG3_TIFF_RGB_16_bit_190562.tif/Dublicated_TIFF_TRANSFORMED_PQ_BT2020_FR_HD_HdM_HDR_2023_VideoDisco_LogC3_AWG3_TIFF_RGB_16_bit_190562.tif_000000_V1-0036.000000.tif")

test03 = c.read_image("/Volumes/T7_Shield/11.1_Source_Content_Scaled/PQ_UHD/13.1_TRANSFORMED_PQ_FR_HD/20231219_HdM_HDR_2023_VideoDisco_V2_Selection/TRANSFORMED_PQ_BT2020_FR_HD_HdM_HDR_2023_VideoDisco_LogC3_AWG3_TIFF_RGB_16_bit_190562/TRANSFORMED_PQ_BT2020_FR_HD_HdM_HDR_2023_VideoDisco_LogC3_AWG3_TIFF_RGB_16_bit_190562_00000000.tif")
test04=c.read_image("/Volumes/JM_SSD/41_TIFF_EXTRACTION/31.1_PQ_UHD/01_ProRes_4444/13.1_TRANSFORMED_PQ_FR_HD/20231219_HdM_HDR_2023_VideoDisco_V2_Selection/TRANSFORMED_PQ_BT2020_FR_HD_HdM_HDR_2023_VideoDisco_LogC3_AWG3_TIFF_RGB_16_bit_190562/TRANSFORMED_PQ_BT2020_FR_HD_HdM_HDR_2023_VideoDisco_LogC3_AWG3_TIFF_RGB_16_bit_190562/Dublicated_TIFF_TRANSFORMED_PQ_BT2020_FR_HD_HdM_HDR_2023_VideoDisco_LogC3_AWG3_TIFF_RGB_16_bit_190562.tif_000000_V1-0036.000000.tif")

In [29]:
np.max(test01)

0.74893934

In [30]:
np.max(test02)

0.79891664

In [31]:
np.max(test03)

0.7980926

In [32]:
np.max(test04)

0.79891664

### Gamut Transform

In [2]:
Matrix_LMS_XYZ = [[1.91020,-1.11212,0.20191],
                  [0.37095,0.62905,0.0],
                  [0.0,0.0,1.0]]

Matrix_BT709_XYZ = [[0.4124564, 0.3575761, 0.1804375],
                    [0.2126729, 0.7151522, 0.0721750],
                    [0.0193339, 0.1191920, 0.9503041]]

Matrix_BT2020_XYZ = [[0.636958, 0.144617, 0.168881],
                    [0.262700, 0.677998, 0.059302],
                    [0.00000, 0.028073, 1.060985]]

Matrix_BT709_BT2020 = [[0.6274, 0.3293, 0.0433],
                        [0.0691, 0.9195, 0.0114],
                        [0.0164, 0.0880, 0.8956]]

Matrix_LogC4_XYZ = [[0.704858320407232064, 0.129760295170463003, 0.115837311473976537],
                        [0.254524176404027025, 0.781477732712002049, -0.036001909116029039],
                        [0.000000000000000000, 0.000000000000000000, 1.089057750759878429]]

Matrix_XYZ_BT2020 = [[1.716651, -0.355671, -0.253366],
                        [-0.666684, 1.616481, 0.015769],
                        [0.017640, -0.042771, 0.942103]]

### LUT Analysis

In [3]:
def LUTGamma(LUT): 
    helper = np.zeros(LUT.size)
    for i in range(LUT.size): 
        helper[i] = np.divide(LUT.table[i , i , i].sum(),3)

    return helper

def LUTGamma_v2(LUT, luma="2020"): 
    helper = np.zeros(LUT.size)
    if luma == "2020": 
        for i in range(LUT.size): 
            r,g,b = LUT.table[i , i , i]
            helper[i] = np.multiply(r,0.2627) + np.multiply(g,0.6780) + np.multiply(b, 0.0593)

        return helper
    if luma == "709": 
        for i in range(LUT.size): 
            r,g,b = LUT.table[i , i , i]
            helper[i] = np.multiply(r,0.2126) + np.multiply(g,0.7152) + np.multiply(b,0.0722)
        return helper
    else: 
        print("Bitte Luma definieren du keck!!! ")

def gray_scale_luma_helper(size=1920):
    gray_scale = np.linspace(0,1,size).reshape((1,size))
    gray_scale = np.repeat(gray_scale[:, :, np.newaxis], 3, axis=2)
    return gray_scale

## Look Up Tables

### False Color

In [9]:

def false_color_jm(img,brightness=1,limit=1000,color=[1,0,0]):
    old_shape = img.shape
    img = img.reshape(-1, 3)
    luma = np.multiply(img[:,0],0.2627) + np.multiply(img[:,1],0.6780) + np.multiply(img[:,2],0.0593)

    luma_mask = luma > c.models.eotf_inverse_ST2084(limit)
    img_false_color = np.copy(img * brightness)

    img_false_color[luma_mask] = color

    print("Max Luma: ", c.models.eotf_ST2084(luma.max()), "Pixels over Limit:", luma_mask.sum())
    return np.reshape(img_false_color, old_shape) 

def multi_false_color_jm_BT2020(img,brightness=1):
    old_shape = img.shape
    img = img.reshape(-1, 3)
    luma = np.multiply(img[:,0],0.2627) + np.multiply(img[:,1],0.6780) + np.multiply(img[:,2],0.0593)

    img_false_color = img * brightness

    luma_mask_100 = luma > c.models.eotf_inverse_ST2084(100)
    img_false_color[luma_mask_100] = [0.5,0,1]

    luma_mask_203 = luma > c.models.eotf_inverse_ST2084(203)
    img_false_color[luma_mask_203] = [0,0.5,0]

    luma_mask_1000 = luma > c.models.eotf_inverse_ST2084(400)
    img_false_color[luma_mask_1000] = [0,1,1]

    luma_mask_2000 = luma > c.models.eotf_inverse_ST2084(800)
    img_false_color[luma_mask_2000] = [1,1,0]

    luma_mask_3000 = luma > c.models.eotf_inverse_ST2084(1600)
    img_false_color[luma_mask_3000] = [1,0.2,0]

    #luma_mask_3500 = luma > c.models.eotf_inverse_ST2084(3500)
    #img_false_color[luma_mask_3500] = [1,0,0]
    
    return np.reshape(img_false_color, old_shape) 

def multi_false_color_jm_Master(img,brightness=1):
    old_shape = img.shape
    img = img.reshape(-1, 3)
    luma = np.multiply(img[:,0],0.2627) + np.multiply(img[:,1],0.6780) + np.multiply(img[:,2],0.0593)

    img_false_color = img

    luma_mask_0 = luma > c.models.eotf_inverse_HLG_BT2100(0,0,1000)
    img_false_color[luma_mask_0] = [0,0,1]

    luma_mask_001 = luma > c.models.eotf_inverse_HLG_BT2100(0.01,0,1000)
    img_false_color[luma_mask_001] = [0.5,0,1]

    luma_mask_01 = luma > c.models.eotf_inverse_HLG_BT2100(0.1,0,1000)
    img_false_color[luma_mask_01] = [1,0,1]

    luma_mask_1 = luma > c.models.eotf_inverse_HLG_BT2100(1,0,1000)
    img_false_color[luma_mask_1] = [0,0.5,1]

    luma_mask_315 = luma > c.models.eotf_inverse_HLG_BT2100(3.125,0,1000)
    img_false_color[luma_mask_315] = [0,1,1]

    luma_mask_125 = luma > c.models.eotf_inverse_HLG_BT2100(12.5,0,1000)
    img_false_color[luma_mask_125] = [0,1,0.5]

    luma_mask_125 = luma > c.models.eotf_inverse_HLG_BT2100(50,0,1000)
    img_false_color[luma_mask_125] = [0,1,0]

    luma_mask_100 = luma > c.models.eotf_inverse_HLG_BT2100(100,0,1000)
    img_false_color[luma_mask_100] = [0.5,1,0]

    luma_mask_400 = luma > c.models.eotf_inverse_HLG_BT2100(400,0,1000)
    img_false_color[luma_mask_400] = [1,1,0]

    luma_mask_800 = luma > c.models.eotf_inverse_HLG_BT2100(800,0,1000)
    img_false_color[luma_mask_800] = [1,0,0]

    #luma_mask_3500 = luma > c.models.eotf_inverse_ST2084(3500)
    #img_false_color[luma_mask_3500] = [1,0,0]
    
    return np.reshape(img_false_color, old_shape) 


def HLG_in_PQ_Container(x,L_B=0,L_W=1000):
    ootf_image = c.models.ootf_HLG_BT2100(c.models.oetf_inverse_HLG_BT2100(x),L_B,L_W)
    pq_image = c.models.eotf_inverse_PQ_BT2100(ootf_image)
    return pq_image

def HLG_Gained_in_PQ_Container(x,L_B=0,L_W=1000):
    ootf_image = c.models.ootf_HLG_BT2100(c.models.oetf_inverse_HLG_BT2100(x),0,1000)
    ootf_image_gained = np.multiply(ootf_image,np.divide(L_W,1000))
    pq_image = c.models.eotf_inverse_PQ_BT2100(ootf_image_gained)
    return pq_image

def HLG_in_Rec_1886_in_PQ_Container(x,L_B=0,L_W=100):
    ootf_image = c.models.eotf_BT1886(x,L_B,L_W)
    pq_image = c.models.eotf_inverse_PQ_BT2100(ootf_image)
    return pq_image

def PQ_in_HLG_Container(x,L_B=0,L_W=1000):
    image_display = c.models.eotf_ST2084(x)
    eotf_inverse = c.models.eotf_inverse_HLG_BT2100(image_display,L_B=L_B,L_W=L_W)
    return eotf_inverse

In [10]:
# Export False Color LUTs

value_brightness = 0.5
Empty_LUT_Container = c.LUT3D(size=(65,65,65))

# Specify Colors and Brightness
# table_100_violet = false_color_jm(Empty_LUT_Container.table,brightness=value_brightness,limit=100,color=[0.5,0,1])
# table_203_green = false_color_jm(Empty_LUT_Container.table,brightness=value_brightness,limit=203,color=[0,0.5,0])
# table_1000_cyan = false_color_jm(Empty_LUT_Container.table,brightness=value_brightness,limit=1000,color=[0,1,1])
# table_2000_yellow = false_color_jm(Empty_LUT_Container.table,brightness=value_brightness,limit=2000,color=[1,1,0])
# table_3000_amber = false_color_jm(Empty_LUT_Container.table,brightness=value_brightness,limit=3000,color=[1,0.3,0])
# table_3500_red = false_color_jm(Empty_LUT_Container.table,brightness=value_brightness,limit=3500,color=[1,0,0])
table_multi = multi_false_color_jm_Master(Empty_LUT_Container.table,brightness=value_brightness)

table_pq_in_hlg = PQ_in_HLG_Container(Empty_LUT_Container.table)

# Generate LUTs
# LUT_table_100_violet = c.LUT3D(table_100_violet)
# LUT_table_203_green = c.LUT3D(table_203_green)
# LUT_table_1000_cyan = c.LUT3D(table_1000_cyan)
# LUT_table_2000_yellow = c.LUT3D(table_2000_yellow)
# LUT_table_3000_amber = c.LUT3D(table_3000_amber)
# LUT_table_3500_red = c.LUT3D(table_3500_red)
LUT_table_multi = c.LUT3D(table_multi)

LUT_table_pq_in_hlg = c.LUT3D(table_pq_in_hlg)

# Export LUTs
# c.write_LUT(LUT_table_100_violet,f"/Users/justus/Desktop/LUT_100_violet_Brightness_{value_brightness}.cube")
# c.write_LUT(LUT_table_203_green,f"/Users/justus/Desktop/LUT_203_green_Brightness_{value_brightness}.cube")
# c.write_LUT(LUT_table_1000_cyan,f"/Users/justus/Desktop/LUT_1000_cyan_Brightness_{value_brightness}.cube")
# c.write_LUT(LUT_table_2000_yellow,f"/Users/justus/Desktop/LUT_2000_yellow_Brightness_{value_brightness}.cube")
# c.write_LUT(LUT_table_3000_amber,f"/Users/justus/Desktop/LUT_3000_amber_Brightness_{value_brightness}.cube")
# c.write_LUT(LUT_table_3500_red,f"/Users/justus/Desktop/LUT_3500_red_Brightness_{value_brightness}.cube")
c.write_LUT(LUT_table_multi,f"/Users/justus/Desktop/LUT_multi_Brightness_{value_brightness}.cube")

c.write_LUT(LUT_table_pq_in_hlg,f"/Users/justus/Desktop/20230928_PQ_in_HLG_EOTF_OOTF_Inverse_jm.cube")

/Users/justus/.local/lib/python3.9/site-packages/colour/utilities/verbose.py:262: ColourUsageWarning: "Recommendation ITU-R BT.2100" "Reference HLG OOTF" uses RGB Luminance in computations and expects a vector input, thus the given input array will be stacked to compose a vector for internal computations but a single component will be output.
  warn(*args, **kwargs)
/Users/justus/.local/lib/python3.9/site-packages/colour/models/rgb/transfer_functions/itur_bt_2100.py:1525: RuntimeWarning: divide by zero encountered in power
  Y_D_alpha = np.abs(Y_D / alpha) ** ((1 - gamma) / gamma)
/Users/justus/.local/lib/python3.9/site-packages/colour/models/rgb/transfer_functions/itur_bt_2100.py:1530: RuntimeWarning: invalid value encountered in multiply
  Y_D_alpha * R_D / alpha,
/Users/justus/.local/lib/python3.9/site-packages/colour/models/rgb/transfer_functions/itur_bt_2100.py:1535: RuntimeWarning: invalid value encountered in multiply
  Y_D_alpha * G_D / alpha,
/Users/justus/.local/lib/python3.9

True

In [ ]:
LUT_NBCU = c.read_LUT(f"/Library/Application Support/Blackmagic Design/DaVinci Resolve/LUT/NBCU/5-NBCU_PQ2SDR_DL_v1.cube")
LUT_NBCU_HLG = c.read_LUT(f"/Library/Application Support/Blackmagic Design/DaVinci Resolve/LUT/NBCU/3-NBCU_HLG2SDR_DL_v1.1.cube")
#LUT_ARRI_4K_PQ_200 = c.read_LUT(f"/Users/justus/Downloads/ARRI_LogC3_v1-2_LUT_Package/LUTs/ARRI_LogC3-to-St2084_4K_Rec2100-D65_DW200_v2_33.cube")
#LUT_ARRI_C4_to_709_D65 = c.read_LUT(f"/Users/justus/Downloads/ARRI_LogC4_v1_LUT_Package/LUTs/ARRI_LogC4-to-Gamma24_Rec709-D65_v1-65.cube")
#LUT_ARRI_C3_to_709_D65 = c.read_LUT(f"/Users/justus/Downloads/ARRI_LogC2Video_709_davinci3d_33.cube")
#LUT_SLOG3_to_709_D65 = c.read_LUT(f"/Users/justus/Downloads/SonyLookProfiles_SLog3_SGamut3Cine/1_SGamut3CineSLog3_To_LC-709.cube")

In [ ]:
Inverse_LUT_NBCU = c.LUT3D.invert(LUT_NBCU)
Inverse_LUT_ARRI_4K_PQ_200 = c.LUT3D.invert(LUT_ARRI_4K_PQ_200)

## Perception Models

### Barten Model

In [ ]:
import colour as c
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import fmin


def Invers_CSF_Barten_BT2246(L,X_0=60,Y_0=60,k=3.0,T=0.1,X_max=12,N_max=15,n=0.03,p=1.2274*10**6,phi_0=3*10**-8,u_0=7):

    def max_sfrequency(L,X_0,Y_0,k,T,X_max,N_max,n,p,phi_0,u_0):
        max_sfrequency = []
        for L_v in L:
            d = c.contrast.pupil_diameter_Barten1999(L_v, X_0)
            sigma = c.contrast.sigma_Barten1999(0.5 / 60, 0.08 / 60, d)
            E = c.contrast.retinal_illuminance_Barten1999(L_v, d, True)
            max_sfrequency.append(
                fmin(
                    lambda x: (
                        -c.contrast.contrast_sensitivity_function_Barten1999(
                            u=x, sigma=sigma,
                            X_0=X_0, Y_0=Y_0,
                            E=E, k=k, T=T,
                            X_max=X_max, N_max=N_max,
                            n=n, p=p, phi_0=phi_0,u_0=u_0
                        )
                    ),
                    0,
                    disp=False,
                )[0]
            )
        return np.array(max_sfrequency)
    
    Y_0 = X_0
    d = c.contrast.pupil_diameter_Barten1999(L, X_0, Y_0)
    sigma = c.contrast.sigma_Barten1999(0.5 / 60, 0.08 / 60, d)
    E = c.contrast.retinal_illuminance_Barten1999(L, d)

    u = max_sfrequency(L,X_0=60,Y_0=60,k=3.0,T=0.1,X_max=12,N_max=15,n=0.03,p=1.2274*10**6,phi_0=3*10**-8,u_0=7)

    return np.multiply(
        np.multiply( 
        np.divide(1 , c.contrast.contrast_sensitivity_function_Barten1999(
        u=u, sigma=sigma, 
        E=E, X_0=X_0, Y_0=Y_0,
        k=k, T=T, X_max=X_max, N_max=N_max,
        n=n, p=p, phi_0=phi_0, u_0=u_0)
        )
        , 2 )
        , np.divide(1 , 1.27))

# x1 = np.logspace(np.log10(0.01), np.log10(100), 100)
# axR.plot(x1,Invers_CSF_Barten_BT2246(np.logspace(np.log10(0.01), np.log10(100), 100),X_0=60),color="magenta")

In [ ]:
import colour as c
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import fmin


def Invers_CSF_Barten_Digital_Cinema(L,X_0=60,Y_0=60,k=3.0,T=0.1,X_max=12,N_max=15,n=0.03,p=1.2274*10**6,phi_0=3*10**-8,u_0=7):

    def max_sfrequency(L,X_0,Y_0,k,T,X_max,N_max,n,p,phi_0,u_0):
        max_sfrequency = []
        for L_v in L:
            d = c.contrast.pupil_diameter_Barten1999(L_v, X_0)
            sigma = c.contrast.sigma_Barten1999(0.5 / 60, 0.08 / 60, d)
            E = c.contrast.retinal_illuminance_Barten1999(L_v, d, True)
            max_sfrequency.append(
                fmin(
                    lambda x: (
                        -c.contrast.contrast_sensitivity_function_Barten1999(
                            u=x, sigma=sigma,
                            X_0=X_0, Y_0=Y_0,
                            E=E, k=k, T=T,
                            X_max=X_max, N_max=N_max,
                            n=n, p=p, phi_0=phi_0,u_0=u_0
                        )
                    ),
                    0,
                    disp=False,
                )[0]
            )
        return np.array(max_sfrequency)
    
    Y_0 = X_0
    d = c.contrast.pupil_diameter_Barten1999(L, X_0, Y_0)
    sigma = c.contrast.sigma_Barten1999(0.5 / 60, 0.08 / 60, d)
    E = c.contrast.retinal_illuminance_Barten1999(L, d)

    u = max_sfrequency(L,X_0=60,Y_0=60,k=3.0,T=0.1,X_max=12,N_max=15,n=0.03,p=1.2274*10**6,phi_0=3*10**-8,u_0=7)

    return np.divide(1 , c.contrast.contrast_sensitivity_function_Barten1999(
        u=u, sigma=sigma, 
        E=E, X_0=X_0, Y_0=Y_0,
        k=k, T=T, X_max=X_max, N_max=N_max,
        n=n, p=p, phi_0=phi_0, u_0=u_0)
        )

## Image Analysis

In [ ]:
x1 = np.linspace(0.000000000000001,1,1000000)

global_counter = np.log2(np.linspace(0.000000000000001,1*59.5208/1.8,1000000))
global_values_r_r = np.round((219*c.models.eotf_inverse_ST2084(c.models.eotf_BT1886(c.models.oetf_BT709(x1*59.5208))*100)+16)*np.power(2,(10-8)),0)

pq_dictionary = {global_counter[i]: global_values[i] for i in range(len(global_counter))}

temp = {val: key for key, val in pq_dictionary.items()}
res = {val: key for key, val in temp.items()}

print(res)

values_lower_minues_10 = [v for k,v in res.items() if k < -10]
values_minues_10_to_9 = [v for k,v in res.items() if -10 <= k and k < -9]
values_minues_9_to_8 = [v for k,v in res.items() if -9 <= k and k < -8]
values_minues_8_to_7 = [v for k,v in res.items() if -8 <= k and k < -7]
values_minues_7_to_6 = [v for k,v in res.items() if -7 <= k and k < -6]
values_minues_6_to_5 = [v for k,v in res.items() if -6 <= k and k < -5]
values_minues_5_to_4 = [v for k,v in res.items() if -5 <= k and k < -4]
values_minues_4_to_3 = [v for k,v in res.items() if -4 <= k and k < -3]
values_minues_3_to_2 = [v for k,v in res.items() if -3 <= k and k < -2]
values_minues_2_to_1 = [v for k,v in res.items() if -2 <= k and k < -1]
values_minues_1_to_0 = [v for k,v in res.items() if -1 <= k and k < -0]
values_plus_0_to_1 = [v for k,v in res.items() if 0 <= k and k < 1]
values_plus_1_to_2 = [v for k,v in res.items() if 1 <= k and k < 2]
values_plus_2_to_3 = [v for k,v in res.items() if 2 <= k and k < 3]
values_plus_3_to_4 = [v for k,v in res.items() if 3 <= k and k < 4]
values_plus_4_to_5 = [v for k,v in res.items() if 4 <= k and k < 5]
values_plus_5_to_6 = [v for k,v in res.items() if 5 <= k and k < 6]
values_over_6 = [v for k,v in res.items() if 6 <= k]

value_counter_pq = [len(values_lower_minues_10),
                 len(values_minues_10_to_9),
                 len(values_minues_9_to_8),
                 len(values_minues_8_to_7),
                 len(values_minues_7_to_6),
                 len(values_minues_6_to_5),
                 len(values_minues_5_to_4),
                 len(values_minues_4_to_3),
                 len(values_minues_3_to_2),
                 len(values_minues_2_to_1),
                 len(values_minues_1_to_0),
                 len(values_plus_0_to_1),
                 len(values_plus_1_to_2),
                 len(values_plus_2_to_3),
                 len(values_plus_3_to_4),
                 len(values_plus_4_to_5),
                 len(values_plus_5_to_6),
                 len(values_over_6)]


global_counter = np.log2(np.linspace(0.000000000000001,1*3.75,1000000))
global_values = np.clip(np.round((219*c.models.oetf_HLG_BT2100(x1)+16)*np.power(2,(10-8)),0),0,940)

pq_dictionary = {global_counter[i]: global_values[i] for i in range(len(global_counter))}

temp = {val: key for key, val in pq_dictionary.items()}
res = {val: key for key, val in temp.items()}

print(res)

values_lower_minues_10 = [v for k,v in res.items() if k < -10]
values_minues_10_to_9 = [v for k,v in res.items() if -10 <= k and k < -9]
values_minues_9_to_8 = [v for k,v in res.items() if -9 <= k and k < -8]
values_minues_8_to_7 = [v for k,v in res.items() if -8 <= k and k < -7]
values_minues_7_to_6 = [v for k,v in res.items() if -7 <= k and k < -6]
values_minues_6_to_5 = [v for k,v in res.items() if -6 <= k and k < -5]
values_minues_5_to_4 = [v for k,v in res.items() if -5 <= k and k < -4]
values_minues_4_to_3 = [v for k,v in res.items() if -4 <= k and k < -3]
values_minues_3_to_2 = [v for k,v in res.items() if -3 <= k and k < -2]
values_minues_2_to_1 = [v for k,v in res.items() if -2 <= k and k < -1]
values_minues_1_to_0 = [v for k,v in res.items() if -1 <= k and k < -0]
values_plus_0_to_1 = [v for k,v in res.items() if 0 <= k and k < 1]
values_plus_1_to_2 = [v for k,v in res.items() if 1 <= k and k < 2]
values_plus_2_to_3 = [v for k,v in res.items() if 2 <= k and k < 3]
values_plus_3_to_4 = [v for k,v in res.items() if 3 <= k and k < 4]
values_plus_4_to_5 = [v for k,v in res.items() if 4 <= k and k < 5]
values_plus_5_to_6 = [v for k,v in res.items() if 5 <= k and k < 6]
values_over_6 = [v for k,v in res.items() if 6 <= k]

value_counter_hlg = [len(values_lower_minues_10),
                 len(values_minues_10_to_9),
                 len(values_minues_9_to_8),
                 len(values_minues_8_to_7),
                 len(values_minues_7_to_6),
                 len(values_minues_6_to_5),
                 len(values_minues_5_to_4),
                 len(values_minues_4_to_3),
                 len(values_minues_3_to_2),
                 len(values_minues_2_to_1),
                 len(values_minues_1_to_0),
                 len(values_plus_0_to_1),
                 len(values_plus_1_to_2),
                 len(values_plus_2_to_3),
                 len(values_plus_3_to_4),
                 len(values_plus_4_to_5),
                 len(values_plus_5_to_6),
                 len(values_over_6)]


###################################################################################

global_counter = np.log2(np.linspace(0.000000000000001,1*3.75*2,1000000))
global_values = np.clip(np.round((219*c.models.oetf_HLG_BT2100(x1*2)+16)*np.power(2,(10-8)),0),0,1022)

pq_dictionary = {global_counter[i]: global_values[i] for i in range(len(global_counter))}

temp = {val: key for key, val in pq_dictionary.items()}
res = {val: key for key, val in temp.items()}

print(res)

values_lower_minues_10 = [v for k,v in res.items() if k < -10]
values_minues_10_to_9 = [v for k,v in res.items() if -10 <= k and k < -9]
values_minues_9_to_8 = [v for k,v in res.items() if -9 <= k and k < -8]
values_minues_8_to_7 = [v for k,v in res.items() if -8 <= k and k < -7]
values_minues_7_to_6 = [v for k,v in res.items() if -7 <= k and k < -6]
values_minues_6_to_5 = [v for k,v in res.items() if -6 <= k and k < -5]
values_minues_5_to_4 = [v for k,v in res.items() if -5 <= k and k < -4]
values_minues_4_to_3 = [v for k,v in res.items() if -4 <= k and k < -3]
values_minues_3_to_2 = [v for k,v in res.items() if -3 <= k and k < -2]
values_minues_2_to_1 = [v for k,v in res.items() if -2 <= k and k < -1]
values_minues_1_to_0 = [v for k,v in res.items() if -1 <= k and k < -0]
values_plus_0_to_1 = [v for k,v in res.items() if 0 <= k and k < 1]
values_plus_1_to_2 = [v for k,v in res.items() if 1 <= k and k < 2]
values_plus_2_to_3 = [v for k,v in res.items() if 2 <= k and k < 3]
values_plus_3_to_4 = [v for k,v in res.items() if 3 <= k and k < 4]
values_plus_4_to_5 = [v for k,v in res.items() if 4 <= k and k < 5]
values_plus_5_to_6 = [v for k,v in res.items() if 5 <= k and k < 6]
values_over_6 = [v for k,v in res.items() if 6 <= k]

value_counter_hlg_plus = [len(values_lower_minues_10),
                 len(values_minues_10_to_9),
                 len(values_minues_9_to_8),
                 len(values_minues_8_to_7),
                 len(values_minues_7_to_6),
                 len(values_minues_6_to_5),
                 len(values_minues_5_to_4),
                 len(values_minues_4_to_3),
                 len(values_minues_3_to_2),
                 len(values_minues_2_to_1),
                 len(values_minues_1_to_0),
                 len(values_plus_0_to_1),
                 len(values_plus_1_to_2),
                 len(values_plus_2_to_3),
                 len(values_plus_3_to_4),
                 len(values_plus_4_to_5),
                 len(values_plus_5_to_6),
                 len(values_over_6)]


#############################################################################

global_counter = np.log2(np.linspace(0.000000000000001,1*c.models.log_decoding_SLog3(1),1000000))
global_values = np.clip(np.round(c.models.log_encoding_SLog3(x1*c.models.log_decoding_SLog3(1))*1023,0),0,940)

pq_dictionary = {global_counter[i]: global_values[i] for i in range(len(global_counter))}

temp = {val: key for key, val in pq_dictionary.items()}
res = {val: key for key, val in temp.items()}

print(res)

values_lower_minues_10 = [v for k,v in res.items() if k < -10]
values_minues_10_to_9 = [v for k,v in res.items() if -10 <= k and k < -9]
values_minues_9_to_8 = [v for k,v in res.items() if -9 <= k and k < -8]
values_minues_8_to_7 = [v for k,v in res.items() if -8 <= k and k < -7]
values_minues_7_to_6 = [v for k,v in res.items() if -7 <= k and k < -6]
values_minues_6_to_5 = [v for k,v in res.items() if -6 <= k and k < -5]
values_minues_5_to_4 = [v for k,v in res.items() if -5 <= k and k < -4]
values_minues_4_to_3 = [v for k,v in res.items() if -4 <= k and k < -3]
values_minues_3_to_2 = [v for k,v in res.items() if -3 <= k and k < -2]
values_minues_2_to_1 = [v for k,v in res.items() if -2 <= k and k < -1]
values_minues_1_to_0 = [v for k,v in res.items() if -1 <= k and k < -0]
values_plus_0_to_1 = [v for k,v in res.items() if 0 <= k and k < 1]
values_plus_1_to_2 = [v for k,v in res.items() if 1 <= k and k < 2]
values_plus_2_to_3 = [v for k,v in res.items() if 2 <= k and k < 3]
values_plus_3_to_4 = [v for k,v in res.items() if 3 <= k and k < 4]
values_plus_4_to_5 = [v for k,v in res.items() if 4 <= k and k < 5]
values_plus_5_to_6 = [v for k,v in res.items() if 5 <= k and k < 6]
values_over_6 = [v for k,v in res.items() if 6 <= k]

value_counter_slog3 = [len(values_lower_minues_10),
                 len(values_minues_10_to_9),
                 len(values_minues_9_to_8),
                 len(values_minues_8_to_7),
                 len(values_minues_7_to_6),
                 len(values_minues_6_to_5),
                 len(values_minues_5_to_4),
                 len(values_minues_4_to_3),
                 len(values_minues_3_to_2),
                 len(values_minues_2_to_1),
                 len(values_minues_1_to_0),
                 len(values_plus_0_to_1),
                 len(values_plus_1_to_2),
                 len(values_plus_2_to_3),
                 len(values_plus_3_to_4),
                 len(values_plus_4_to_5),
                 len(values_plus_5_to_6),
                 len(values_over_6)]

print(value_counter_slog3)

#############################################################################

global_counter = np.log2(np.linspace(0.000000000000001,1*c.models.log_decoding_SLog3(1),1000000))
global_values = np.clip(np.round(c.models.log_encoding_SLog3(x1*c.models.log_decoding_SLog3(1))*1023,0),0,1022)

pq_dictionary = {global_counter[i]: global_values[i] for i in range(len(global_counter))}

temp = {val: key for key, val in pq_dictionary.items()}
res = {val: key for key, val in temp.items()}

print(res)

values_lower_minues_10 = [v for k,v in res.items() if k < -10]
values_minues_10_to_9 = [v for k,v in res.items() if -10 <= k and k < -9]
values_minues_9_to_8 = [v for k,v in res.items() if -9 <= k and k < -8]
values_minues_8_to_7 = [v for k,v in res.items() if -8 <= k and k < -7]
values_minues_7_to_6 = [v for k,v in res.items() if -7 <= k and k < -6]
values_minues_6_to_5 = [v for k,v in res.items() if -6 <= k and k < -5]
values_minues_5_to_4 = [v for k,v in res.items() if -5 <= k and k < -4]
values_minues_4_to_3 = [v for k,v in res.items() if -4 <= k and k < -3]
values_minues_3_to_2 = [v for k,v in res.items() if -3 <= k and k < -2]
values_minues_2_to_1 = [v for k,v in res.items() if -2 <= k and k < -1]
values_minues_1_to_0 = [v for k,v in res.items() if -1 <= k and k < -0]
values_plus_0_to_1 = [v for k,v in res.items() if 0 <= k and k < 1]
values_plus_1_to_2 = [v for k,v in res.items() if 1 <= k and k < 2]
values_plus_2_to_3 = [v for k,v in res.items() if 2 <= k and k < 3]
values_plus_3_to_4 = [v for k,v in res.items() if 3 <= k and k < 4]
values_plus_4_to_5 = [v for k,v in res.items() if 4 <= k and k < 5]
values_plus_5_to_6 = [v for k,v in res.items() if 5 <= k and k < 6]
values_over_6 = [v for k,v in res.items() if 6 <= k]

value_counter_slog3_plus = [len(values_lower_minues_10),
                 len(values_minues_10_to_9),
                 len(values_minues_9_to_8),
                 len(values_minues_8_to_7),
                 len(values_minues_7_to_6),
                 len(values_minues_6_to_5),
                 len(values_minues_5_to_4),
                 len(values_minues_4_to_3),
                 len(values_minues_3_to_2),
                 len(values_minues_2_to_1),
                 len(values_minues_1_to_0),
                 len(values_plus_0_to_1),
                 len(values_plus_1_to_2),
                 len(values_plus_2_to_3),
                 len(values_plus_3_to_4),
                 len(values_plus_4_to_5),
                 len(values_plus_5_to_6),
                 len(values_over_6)]

print(value_counter_slog3_plus)

########################################################################################

global_counter = np.log2(np.linspace(0.000000000000001,1,1000000))
global_values = np.round(c.models.oetf_BT709(x1)*1023,0)

pq_dictionary = {global_counter[i]: global_values[i] for i in range(len(global_counter))}

temp = {val: key for key, val in pq_dictionary.items()}
res = {val: key for key, val in temp.items()}

#print(res)

values_lower_minues_10 = [v for k,v in res.items() if k < -10]
values_minues_10_to_9 = [v for k,v in res.items() if -10 <= k and k < -9]
values_minues_9_to_8 = [v for k,v in res.items() if -9 <= k and k < -8]
values_minues_8_to_7 = [v for k,v in res.items() if -8 <= k and k < -7]
values_minues_7_to_6 = [v for k,v in res.items() if -7 <= k and k < -6]
values_minues_6_to_5 = [v for k,v in res.items() if -6 <= k and k < -5]
values_minues_5_to_4 = [v for k,v in res.items() if -5 <= k and k < -4]
values_minues_4_to_3 = [v for k,v in res.items() if -4 <= k and k < -3]
values_minues_3_to_2 = [v for k,v in res.items() if -3 <= k and k < -2]
values_minues_2_to_1 = [v for k,v in res.items() if -2 <= k and k < -1]
values_minues_1_to_0 = [v for k,v in res.items() if -1 <= k and k < -0]
values_plus_0_to_1 = [v for k,v in res.items() if 0 <= k and k < 1]
values_plus_1_to_2 = [v for k,v in res.items() if 1 <= k and k < 2]
values_plus_2_to_3 = [v for k,v in res.items() if 2 <= k and k < 3]
values_plus_3_to_4 = [v for k,v in res.items() if 3 <= k and k < 4]
values_plus_4_to_5 = [v for k,v in res.items() if 4 <= k and k < 5]
values_plus_5_to_6 = [v for k,v in res.items() if 5 <= k and k < 6]
values_over_6 = [v for k,v in res.items() if 6 <= k]

value_counter_sdr = [len(values_lower_minues_10),
                 len(values_minues_10_to_9),
                 len(values_minues_9_to_8),
                 len(values_minues_8_to_7),
                 len(values_minues_7_to_6),
                 len(values_minues_6_to_5),
                 len(values_minues_5_to_4),
                 len(values_minues_4_to_3),
                 len(values_minues_3_to_2),
                 len(values_minues_2_to_1),
                 len(values_minues_1_to_0),
                 len(values_plus_0_to_1),
                 len(values_plus_1_to_2),
                 len(values_plus_2_to_3),
                 len(values_plus_3_to_4),
                 len(values_plus_4_to_5),
                 len(values_plus_5_to_6),
                 len(values_over_6)]



#############################################################################

global_counter = np.log2(np.linspace(0.000000000000001,1*c.models.log_decoding_ALEXALogC(1),1000000))
global_values = np.clip(np.round(c.models.log_encoding_ALEXALogC(x1*c.models.log_decoding_ALEXALogC(1))*1023,0),0,940)

pq_dictionary = {global_counter[i]: global_values[i] for i in range(len(global_counter))}

temp = {val: key for key, val in pq_dictionary.items()}
res = {val: key for key, val in temp.items()}

print(res)

values_lower_minues_10 = [v for k,v in res.items() if k < -10]
values_minues_10_to_9 = [v for k,v in res.items() if -10 <= k and k < -9]
values_minues_9_to_8 = [v for k,v in res.items() if -9 <= k and k < -8]
values_minues_8_to_7 = [v for k,v in res.items() if -8 <= k and k < -7]
values_minues_7_to_6 = [v for k,v in res.items() if -7 <= k and k < -6]
values_minues_6_to_5 = [v for k,v in res.items() if -6 <= k and k < -5]
values_minues_5_to_4 = [v for k,v in res.items() if -5 <= k and k < -4]
values_minues_4_to_3 = [v for k,v in res.items() if -4 <= k and k < -3]
values_minues_3_to_2 = [v for k,v in res.items() if -3 <= k and k < -2]
values_minues_2_to_1 = [v for k,v in res.items() if -2 <= k and k < -1]
values_minues_1_to_0 = [v for k,v in res.items() if -1 <= k and k < -0]
values_plus_0_to_1 = [v for k,v in res.items() if 0 <= k and k < 1]
values_plus_1_to_2 = [v for k,v in res.items() if 1 <= k and k < 2]
values_plus_2_to_3 = [v for k,v in res.items() if 2 <= k and k < 3]
values_plus_3_to_4 = [v for k,v in res.items() if 3 <= k and k < 4]
values_plus_4_to_5 = [v for k,v in res.items() if 4 <= k and k < 5]
values_plus_5_to_6 = [v for k,v in res.items() if 5 <= k and k < 6]
values_over_6 = [v for k,v in res.items() if 6 <= k]

value_counter_logc3 = [len(values_lower_minues_10),
                 len(values_minues_10_to_9),
                 len(values_minues_9_to_8),
                 len(values_minues_8_to_7),
                 len(values_minues_7_to_6),
                 len(values_minues_6_to_5),
                 len(values_minues_5_to_4),
                 len(values_minues_4_to_3),
                 len(values_minues_3_to_2),
                 len(values_minues_2_to_1),
                 len(values_minues_1_to_0),
                 len(values_plus_0_to_1),
                 len(values_plus_1_to_2),
                 len(values_plus_2_to_3),
                 len(values_plus_3_to_4),
                 len(values_plus_4_to_5),
                 len(values_plus_5_to_6),
                 len(values_over_6)]

print(value_counter_logc3)


#############################################################################

global_counter = np.log2(np.linspace(0.000000000000001,1*c.models.log_decoding_ALEXALogC(1),1000000))
global_values = np.clip(np.round(c.models.log_encoding_ALEXALogC(x1*c.models.log_decoding_ALEXALogC(1))*1023,0),0,1022)

pq_dictionary = {global_counter[i]: global_values[i] for i in range(len(global_counter))}

temp = {val: key for key, val in pq_dictionary.items()}
res = {val: key for key, val in temp.items()}

print(res)

values_lower_minues_10 = [v for k,v in res.items() if k < -10]
values_minues_10_to_9 = [v for k,v in res.items() if -10 <= k and k < -9]
values_minues_9_to_8 = [v for k,v in res.items() if -9 <= k and k < -8]
values_minues_8_to_7 = [v for k,v in res.items() if -8 <= k and k < -7]
values_minues_7_to_6 = [v for k,v in res.items() if -7 <= k and k < -6]
values_minues_6_to_5 = [v for k,v in res.items() if -6 <= k and k < -5]
values_minues_5_to_4 = [v for k,v in res.items() if -5 <= k and k < -4]
values_minues_4_to_3 = [v for k,v in res.items() if -4 <= k and k < -3]
values_minues_3_to_2 = [v for k,v in res.items() if -3 <= k and k < -2]
values_minues_2_to_1 = [v for k,v in res.items() if -2 <= k and k < -1]
values_minues_1_to_0 = [v for k,v in res.items() if -1 <= k and k < -0]
values_plus_0_to_1 = [v for k,v in res.items() if 0 <= k and k < 1]
values_plus_1_to_2 = [v for k,v in res.items() if 1 <= k and k < 2]
values_plus_2_to_3 = [v for k,v in res.items() if 2 <= k and k < 3]
values_plus_3_to_4 = [v for k,v in res.items() if 3 <= k and k < 4]
values_plus_4_to_5 = [v for k,v in res.items() if 4 <= k and k < 5]
values_plus_5_to_6 = [v for k,v in res.items() if 5 <= k and k < 6]
values_over_6 = [v for k,v in res.items() if 6 <= k]

value_counter_logc3_plus = [len(values_lower_minues_10),
                 len(values_minues_10_to_9),
                 len(values_minues_9_to_8),
                 len(values_minues_8_to_7),
                 len(values_minues_7_to_6),
                 len(values_minues_6_to_5),
                 len(values_minues_5_to_4),
                 len(values_minues_4_to_3),
                 len(values_minues_3_to_2),
                 len(values_minues_2_to_1),
                 len(values_minues_1_to_0),
                 len(values_plus_0_to_1),
                 len(values_plus_1_to_2),
                 len(values_plus_2_to_3),
                 len(values_plus_3_to_4),
                 len(values_plus_4_to_5),
                 len(values_plus_5_to_6),
                 len(values_over_6)]

print(value_counter_logc3_plus)

#############################################################################

global_counter = np.log2(np.linspace(0.000000000000001,1*Decoding_LogC4(1),1000000))
global_values = np.clip(np.round(Encoding_LogC4(x1*Decoding_LogC4(1))*1023,0),0,940)

pq_dictionary = {global_counter[i]: global_values[i] for i in range(len(global_counter))}

temp = {val: key for key, val in pq_dictionary.items()}
res = {val: key for key, val in temp.items()}

print(res)


values_lower_minues_10 = [v for k,v in res.items() if k < -10]
values_minues_10_to_9 = [v for k,v in res.items() if -10 <= k and k < -9]
values_minues_9_to_8 = [v for k,v in res.items() if -9 <= k and k < -8]
values_minues_8_to_7 = [v for k,v in res.items() if -8 <= k and k < -7]
values_minues_7_to_6 = [v for k,v in res.items() if -7 <= k and k < -6]
values_minues_6_to_5 = [v for k,v in res.items() if -6 <= k and k < -5]
values_minues_5_to_4 = [v for k,v in res.items() if -5 <= k and k < -4]
values_minues_4_to_3 = [v for k,v in res.items() if -4 <= k and k < -3]
values_minues_3_to_2 = [v for k,v in res.items() if -3 <= k and k < -2]
values_minues_2_to_1 = [v for k,v in res.items() if -2 <= k and k < -1]
values_minues_1_to_0 = [v for k,v in res.items() if -1 <= k and k < -0]
values_plus_0_to_1 = [v for k,v in res.items() if 0 <= k and k < 1]
values_plus_1_to_2 = [v for k,v in res.items() if 1 <= k and k < 2]
values_plus_2_to_3 = [v for k,v in res.items() if 2 <= k and k < 3]
values_plus_3_to_4 = [v for k,v in res.items() if 3 <= k and k < 4]
values_plus_4_to_5 = [v for k,v in res.items() if 4 <= k and k < 5]
values_plus_5_to_6 = [v for k,v in res.items() if 5 <= k and k < 6]
values_plus_6_to_7 = [v for k,v in res.items() if 6 <= k and k < 7]
values_over_7 = [v for k,v in res.items() if 7 <= k]

value_counter_logc4 = [len(values_lower_minues_10),
                 len(values_minues_10_to_9),
                 len(values_minues_9_to_8),
                 len(values_minues_8_to_7),
                 len(values_minues_7_to_6),
                 len(values_minues_6_to_5),
                 len(values_minues_5_to_4),
                 len(values_minues_4_to_3),
                 len(values_minues_3_to_2),
                 len(values_minues_2_to_1),
                 len(values_minues_1_to_0),
                 len(values_plus_0_to_1),
                 len(values_plus_1_to_2),
                 len(values_plus_2_to_3),
                 len(values_plus_3_to_4),
                 len(values_plus_4_to_5),
                 len(values_plus_5_to_6),
                 len(values_plus_6_to_7),
                 len(values_over_7)]

print(value_counter_logc4)

########################################################################################

#############################################################################

global_counter = np.log2(np.linspace(0.000000000000001,1*Decoding_LogC4(1),1000000))
global_values = np.clip(np.round(Encoding_LogC4(x1*Decoding_LogC4(1))*1023,0),0,1022)

pq_dictionary = {global_counter[i]: global_values[i] for i in range(len(global_counter))}

temp = {val: key for key, val in pq_dictionary.items()}
res = {val: key for key, val in temp.items()}

print(res)

values_lower_minues_10 = [v for k,v in res.items() if k < -10]
values_minues_10_to_9 = [v for k,v in res.items() if -10 <= k and k < -9]
values_minues_9_to_8 = [v for k,v in res.items() if -9 <= k and k < -8]
values_minues_8_to_7 = [v for k,v in res.items() if -8 <= k and k < -7]
values_minues_7_to_6 = [v for k,v in res.items() if -7 <= k and k < -6]
values_minues_6_to_5 = [v for k,v in res.items() if -6 <= k and k < -5]
values_minues_5_to_4 = [v for k,v in res.items() if -5 <= k and k < -4]
values_minues_4_to_3 = [v for k,v in res.items() if -4 <= k and k < -3]
values_minues_3_to_2 = [v for k,v in res.items() if -3 <= k and k < -2]
values_minues_2_to_1 = [v for k,v in res.items() if -2 <= k and k < -1]
values_minues_1_to_0 = [v for k,v in res.items() if -1 <= k and k < -0]
values_plus_0_to_1 = [v for k,v in res.items() if 0 <= k and k < 1]
values_plus_1_to_2 = [v for k,v in res.items() if 1 <= k and k < 2]
values_plus_2_to_3 = [v for k,v in res.items() if 2 <= k and k < 3]
values_plus_3_to_4 = [v for k,v in res.items() if 3 <= k and k < 4]
values_plus_4_to_5 = [v for k,v in res.items() if 4 <= k and k < 5]
values_plus_5_to_6 = [v for k,v in res.items() if 5 <= k and k < 6]
values_plus_6_to_7 = [v for k,v in res.items() if 6 <= k and k < 7]
values_over_7 = [v for k,v in res.items() if 7 <= k]

value_counter_logc4_plus = [len(values_lower_minues_10),
                 len(values_minues_10_to_9),
                 len(values_minues_9_to_8),
                 len(values_minues_8_to_7),
                 len(values_minues_7_to_6),
                 len(values_minues_6_to_5),
                 len(values_minues_5_to_4),
                 len(values_minues_4_to_3),
                 len(values_minues_3_to_2),
                 len(values_minues_2_to_1),
                 len(values_minues_1_to_0),
                 len(values_plus_0_to_1),
                 len(values_plus_1_to_2),
                 len(values_plus_2_to_3),
                 len(values_plus_3_to_4),
                 len(values_plus_4_to_5),
                 len(values_plus_5_to_6),
                 len(values_plus_6_to_7),
                 len(values_over_7)]

print(value_counter_logc4_plus)

########################################################################################



###################################################################################

global_counter = np.log2(np.linspace(0.000000000000001,1,1000000))
global_values = np.clip(np.round((219*c.models.oetf_BT709(x1)+16)*np.power(2,(10-8)),0),0,1022)

bt709_dictionary = {global_counter[i]: global_values[i] for i in range(len(global_counter))}

temp = {val: key for key, val in bt709_dictionary.items()}
res = {val: key for key, val in temp.items()}

print(res)

values_lower_minues_10 = [v for k,v in res.items() if k < -10]
values_minues_10_to_9 = [v for k,v in res.items() if -10 <= k and k < -9]
values_minues_9_to_8 = [v for k,v in res.items() if -9 <= k and k < -8]
values_minues_8_to_7 = [v for k,v in res.items() if -8 <= k and k < -7]
values_minues_7_to_6 = [v for k,v in res.items() if -7 <= k and k < -6]
values_minues_6_to_5 = [v for k,v in res.items() if -6 <= k and k < -5]
values_minues_5_to_4 = [v for k,v in res.items() if -5 <= k and k < -4]
values_minues_4_to_3 = [v for k,v in res.items() if -4 <= k and k < -3]
values_minues_3_to_2 = [v for k,v in res.items() if -3 <= k and k < -2]
values_minues_2_to_1 = [v for k,v in res.items() if -2 <= k and k < -1]
values_minues_1_to_0 = [v for k,v in res.items() if -1 <= k and k < -0]
values_plus_0_to_1 = [v for k,v in res.items() if 0 <= k and k < 1]
values_plus_1_to_2 = [v for k,v in res.items() if 1 <= k and k < 2]
values_plus_2_to_3 = [v for k,v in res.items() if 2 <= k and k < 3]
values_plus_3_to_4 = [v for k,v in res.items() if 3 <= k and k < 4]
values_plus_4_to_5 = [v for k,v in res.items() if 4 <= k and k < 5]
values_plus_5_to_6 = [v for k,v in res.items() if 5 <= k and k < 6]
values_over_6 = [v for k,v in res.items() if 6 <= k]

value_counter_709 = [len(values_lower_minues_10),
                 len(values_minues_10_to_9),
                 len(values_minues_9_to_8),
                 len(values_minues_8_to_7),
                 len(values_minues_7_to_6),
                 len(values_minues_6_to_5),
                 len(values_minues_5_to_4),
                 len(values_minues_4_to_3),
                 len(values_minues_3_to_2),
                 len(values_minues_2_to_1),
                 len(values_minues_1_to_0),
                 len(values_plus_0_to_1),
                 len(values_plus_1_to_2),
                 len(values_plus_2_to_3),
                 len(values_plus_3_to_4),
                 len(values_plus_4_to_5),
                 len(values_plus_5_to_6),
                 len(values_over_6)]


###################################################################################

global_counter = np.log2(np.linspace(0.000000000000001,1,1000000))
global_values = np.clip(np.round((219*c.models.oetf_BT709(x1*2)+16)*np.power(2,(10-8)),0),0,1022)

bt709sw_dictionary = {global_counter[i]: global_values[i] for i in range(len(global_counter))}

temp = {val: key for key, val in bt709sw_dictionary.items()}
res = {val: key for key, val in temp.items()}

print(res)

values_lower_minues_10 = [v for k,v in res.items() if k < -10]
values_minues_10_to_9 = [v for k,v in res.items() if -10 <= k and k < -9]
values_minues_9_to_8 = [v for k,v in res.items() if -9 <= k and k < -8]
values_minues_8_to_7 = [v for k,v in res.items() if -8 <= k and k < -7]
values_minues_7_to_6 = [v for k,v in res.items() if -7 <= k and k < -6]
values_minues_6_to_5 = [v for k,v in res.items() if -6 <= k and k < -5]
values_minues_5_to_4 = [v for k,v in res.items() if -5 <= k and k < -4]
values_minues_4_to_3 = [v for k,v in res.items() if -4 <= k and k < -3]
values_minues_3_to_2 = [v for k,v in res.items() if -3 <= k and k < -2]
values_minues_2_to_1 = [v for k,v in res.items() if -2 <= k and k < -1]
values_minues_1_to_0 = [v for k,v in res.items() if -1 <= k and k < -0]
values_plus_0_to_1 = [v for k,v in res.items() if 0 <= k and k < 1]
values_plus_1_to_2 = [v for k,v in res.items() if 1 <= k and k < 2]
values_plus_2_to_3 = [v for k,v in res.items() if 2 <= k and k < 3]
values_plus_3_to_4 = [v for k,v in res.items() if 3 <= k and k < 4]
values_plus_4_to_5 = [v for k,v in res.items() if 4 <= k and k < 5]
values_plus_5_to_6 = [v for k,v in res.items() if 5 <= k and k < 6]
values_over_6 = [v for k,v in res.items() if 6 <= k]

value_counter_709sw = [len(values_lower_minues_10),
                 len(values_minues_10_to_9),
                 len(values_minues_9_to_8),
                 len(values_minues_8_to_7),
                 len(values_minues_7_to_6),
                 len(values_minues_6_to_5),
                 len(values_minues_5_to_4),
                 len(values_minues_4_to_3),
                 len(values_minues_3_to_2),
                 len(values_minues_2_to_1),
                 len(values_minues_1_to_0),
                 len(values_plus_0_to_1),
                 len(values_plus_1_to_2),
                 len(values_plus_2_to_3),
                 len(values_plus_3_to_4),
                 len(values_plus_4_to_5),
                 len(values_plus_5_to_6),
                 len(values_over_6)]


###################################################################################

global_counter = np.log2(np.linspace(0.000000000000001,1,1000000))
global_values = np.clip(np.round((219*c.models.oetf_BT709(x1*2)+16)*np.power(2,(10-8)),0),0,1022)
global_values = np.clip(np.round(np.divide(global_values,4),0),0,255)

bt7098_dictionary = {global_counter[i]: global_values[i] for i in range(len(global_counter))}

temp = {val: key for key, val in bt7098_dictionary.items()}
res = {val: key for key, val in temp.items()}

print(res)

values_lower_minues_10 = [v for k,v in res.items() if k < -10]
values_minues_10_to_9 = [v for k,v in res.items() if -10 <= k and k < -9]
values_minues_9_to_8 = [v for k,v in res.items() if -9 <= k and k < -8]
values_minues_8_to_7 = [v for k,v in res.items() if -8 <= k and k < -7]
values_minues_7_to_6 = [v for k,v in res.items() if -7 <= k and k < -6]
values_minues_6_to_5 = [v for k,v in res.items() if -6 <= k and k < -5]
values_minues_5_to_4 = [v for k,v in res.items() if -5 <= k and k < -4]
values_minues_4_to_3 = [v for k,v in res.items() if -4 <= k and k < -3]
values_minues_3_to_2 = [v for k,v in res.items() if -3 <= k and k < -2]
values_minues_2_to_1 = [v for k,v in res.items() if -2 <= k and k < -1]
values_minues_1_to_0 = [v for k,v in res.items() if -1 <= k and k < -0]
values_plus_0_to_1 = [v for k,v in res.items() if 0 <= k and k < 1]
values_plus_1_to_2 = [v for k,v in res.items() if 1 <= k and k < 2]
values_plus_2_to_3 = [v for k,v in res.items() if 2 <= k and k < 3]
values_plus_3_to_4 = [v for k,v in res.items() if 3 <= k and k < 4]
values_plus_4_to_5 = [v for k,v in res.items() if 4 <= k and k < 5]
values_plus_5_to_6 = [v for k,v in res.items() if 5 <= k and k < 6]
values_over_6 = [v for k,v in res.items() if 6 <= k]

value_counter_7098 = [len(values_lower_minues_10),
                 len(values_minues_10_to_9),
                 len(values_minues_9_to_8),
                 len(values_minues_8_to_7),
                 len(values_minues_7_to_6),
                 len(values_minues_6_to_5),
                 len(values_minues_5_to_4),
                 len(values_minues_4_to_3),
                 len(values_minues_3_to_2),
                 len(values_minues_2_to_1),
                 len(values_minues_1_to_0),
                 len(values_plus_0_to_1),
                 len(values_plus_1_to_2),
                 len(values_plus_2_to_3),
                 len(values_plus_3_to_4),
                 len(values_plus_4_to_5),
                 len(values_plus_5_to_6),
                 len(values_over_6)]


# ############################################################################



In [8]:
    def full2legal(x):
        return (219*(x)+16)*np.power(2,(10-8))

    def legal2full(x):
        return (((x/np.power(2,(10-8)))-16)/219)

    def lin2log2(x):
        return np.log2(x)

    def log22lin(x):
        return np.power(2,x) * 100

In [ ]:
for i in range(24,61):

    figR, (axI, axV) = plt.subplots(2, 1, figsize=(16, 16), constrained_layout=True, gridspec_kw={'height_ratios': [2, 1]})

    file = f"/Volumes/master/08_DaVinci/Auswahl_A/HDR_Stills_{i}.tiff"
    image_rgb = c.read_image(file)


    # axI_Scene_Linear = c.models.log_decoding_ALEXALogC(c.LUT3D.apply(Inverse_LUT_ARRI_4K_PQ_200,image_rgb))
    axI_Scene_Linear = c.models.log_decoding_ALEXALogC(c.LUT3D.apply(Inverse_LUT_ARRI_4K_PQ_200,image_rgb))

    axI_Scene_Linear_Line_r = axI_Scene_Linear[:,:,0].reshape(-1, 1) * 100
    axI_Scene_Linear_Line_g = axI_Scene_Linear[:,:,1].reshape(-1, 1) * 100
    axI_Scene_Linear_Line_b = axI_Scene_Linear[:,:,2].reshape(-1, 1) * 100

    value_counter_image_r = np.zeros((34))
    value_counter_image_g = np.zeros((34))
    value_counter_image_b = np.zeros((34))

    axI.imshow(c.LUT3D.apply(LUT_NBCU,image_rgb))
    # axI.text(5,1050,f"HDR-SDR-Transform with 5-NBCU_PQ2SDR_DL_v1.cube", color='red', fontsize=10, fontstyle='normal')


    axV.set_ylabel('Amount of Code Values Used [CV]')

    def full2legal(x):
        return (219*(x)+16)*np.power(2,(10-8))

    def legal2full(x):
        return (((x/np.power(2,(10-8)))-16)/219)

    def lin2log2(x):
        return np.log2(x)

    def log22lin(x):
        return np.power(2,x) * 100

    #x1 = np.linspace(0.000000000000001,1,1000000)

    axV.vlines(0.5,0,1000000,color='gray', linestyle="solid", alpha=0.4)
    axV.vlines(1.5,0,1000000,color='gray', linestyle="solid", alpha=0.4)
    axV.vlines(2.5,0,1000000,color='gray', linestyle="solid", alpha=0.4)
    axV.vlines(3.5,0,1000000,color='black', linestyle="solid", alpha=0.5,linewidth=2)
    axV.vlines(4.5,0,1000000,color='black', linestyle="solid", alpha=0.4)
    axV.vlines(5.5,0,1000000,color='black', linestyle="solid", alpha=0.4)
    axV.vlines(6.5,0,1000000,color='black', linestyle="solid", alpha=0.4)
    axV.vlines(7.5,0,1000000,color='black', linestyle="solid", alpha=0.4)
    axV.vlines(8.5,0,1000000,color='black', linestyle="solid", alpha=0.4)
    axV.vlines(9.5,0,1000000,color='black', linestyle="solid", alpha=0.4)
    axV.vlines(10.5,0,1000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
    axV.vlines(11.5,0,1000000,color='gray', linestyle="solid", alpha=0.4)
    axV.vlines(12.5,0,1000000,color='gray', linestyle="solid", alpha=0.4)
    axV.vlines(13.5,0,1000000,color='gray', linestyle="solid", alpha=0.4)
    axV.vlines(14.5,0,1000000,color='gray', linestyle="solid", alpha=0.4)
    axV.vlines(15.5,0,1000000,color='gray', linestyle="solid", alpha=0.4)
    axV.vlines(16.5,0,1000000,color='gray', linestyle="solid", alpha=0.4)
    axV.vlines(17.5,0,1000000,color='gray', linestyle="solid", alpha=0.4)



    axV.vlines(1,0,1000000,color='gray', linestyle="dashed", alpha=0.4)
    axV.vlines(2,0,1000000,color='gray', linestyle="dashed", alpha=0.4)
    axV.vlines(3,0,1000000,color='gray', linestyle="dashed", alpha=0.4)
    axV.vlines(4,0,1000000,color='black', linestyle="dashed", alpha=0.4)
    axV.vlines(5,0,1000000,color='black', linestyle="dashed", alpha=0.4)
    axV.vlines(6,0,1000000,color='black', linestyle="dashed", alpha=0.4)
    axV.vlines(7,0,1000000,color='black', linestyle="dashed", alpha=0.4)
    axV.vlines(8,0,1000000,color='black', linestyle="dashed", alpha=0.4)
    axV.vlines(9,0,1000000,color='black', linestyle="dashed", alpha=0.4)
    axV.vlines(10,0,1000000,color='black', linestyle="dashed", alpha=0.4)
    axV.vlines(11,0,1000000,color='gray', linestyle="dashed", alpha=0.4)
    axV.vlines(12,0,1000000,color='gray', linestyle="dashed", alpha=0.4)
    axV.vlines(13,0,1000000,color='gray', linestyle="dashed", alpha=0.4)
    axV.vlines(14,0,1000000,color='gray', linestyle="dashed", alpha=0.4)
    axV.vlines(15,0,1000000,color='gray', linestyle="dashed", alpha=0.4)
    axV.vlines(16,0,1000000,color='gray', linestyle="dashed", alpha=0.4)
    axV.vlines(17,0,1000000,color='gray', linestyle="dashed", alpha=0.4)



    global_values = axI_Scene_Linear_Line_r[:,0]

    values_lower_minus_10 = list(filter(lambda x: x < log22lin(float(-10)),global_values))
    values_minus_10_to_9_5 = list(filter(lambda x: log22lin(float(-10)) <= x and x < log22lin(float(-9.5)),global_values))
    values_minus_9_5_to_9 = list(filter(lambda x: log22lin(float(-9.5)) <= x and x < log22lin(float(-9)),global_values))
    values_minus_9_to_8_5 = list(filter(lambda x: log22lin(float(-9)) <= x and x < log22lin(float(-8.5)),global_values))
    values_minus_8_5_to_8 = list(filter(lambda x: log22lin(float(-8.5)) <= x and x < log22lin(float(-8)),global_values))
    values_minus_8_to_7_5 = list(filter(lambda x: log22lin(float(-8)) <= x and x < log22lin(float(-7.5)),global_values))
    values_minus_7_5_to_7 = list(filter(lambda x: log22lin(float(-7.5)) <= x and x < log22lin(float(-7)),global_values))
    values_minus_7_to_6_5 = list(filter(lambda x: log22lin(float(-7)) <= x and x < log22lin(float(-6.5)),global_values))
    values_minus_6_5_to_6 = list(filter(lambda x: log22lin(float(-6.5)) <= x and x < log22lin(float(-6)),global_values))
    values_minus_6_to_5_5 = list(filter(lambda x: log22lin(float(-6)) <= x and x < log22lin(float(-5.5)),global_values))
    values_minus_5_5_to_5 = list(filter(lambda x: log22lin(float(-5.5)) <= x and x < log22lin(float(-5)),global_values))
    values_minus_5_to_4_5 = list(filter(lambda x: log22lin(float(-5)) <= x and x < log22lin(float(-4.5)),global_values))
    values_minus_4_5_to_4 = list(filter(lambda x: log22lin(float(-4.5)) <= x and x < log22lin(float(-4)),global_values))
    values_minus_4_to_3_5 = list(filter(lambda x: log22lin(float(-4)) <= x and x < log22lin(float(-3.5)),global_values))
    values_minus_3_5_to_3 = list(filter(lambda x: log22lin(float(-3.5)) <= x and x < log22lin(float(-3)),global_values))
    values_minus_3_to_2_5 = list(filter(lambda x: log22lin(float(-3)) <= x and x < log22lin(float(-2.5)),global_values))
    values_minus_2_5_to_2 = list(filter(lambda x: log22lin(float(-2.5)) <= x and x < log22lin(float(-2)),global_values))
    values_minus_2_to_1_5 = list(filter(lambda x: log22lin(float(-2)) <= x and x < log22lin(float(-1.5)),global_values))
    values_minus_1_5_to_1 = list(filter(lambda x: log22lin(float(-1.5)) <= x and x < log22lin(float(-1)),global_values))
    values_minus_1_to_0_5 = list(filter(lambda x: log22lin(float(-1)) <= x and x < log22lin(float(-0.5)),global_values))
    values_minus_0_5_to_0 = list(filter(lambda x: log22lin(float(-0.5)) <= x and x < log22lin(float(0)),global_values))
    values_plus_0_to_0_5 = list(filter(lambda x: log22lin(float(0)) <= x and x < log22lin(float(0.5)),global_values))
    values_plus_0_5_to_1 = list(filter(lambda x: log22lin(float(0.5)) <= x and x < log22lin(float(1)),global_values))
    values_plus_1_to_1_5 = list(filter(lambda x: log22lin(float(1)) <= x and x < log22lin(float(1.5)),global_values))
    values_plus_1_5_to_2 = list(filter(lambda x: log22lin(float(1.5)) <= x and x < log22lin(float(2)),global_values))
    values_plus_2_to_2_5 = list(filter(lambda x: log22lin(float(2)) <= x and x < log22lin(float(2.5)),global_values))
    values_plus_2_5_to_3 = list(filter(lambda x: log22lin(float(2.5)) <= x and x < log22lin(float(3)),global_values))
    values_plus_3_to_3_5 = list(filter(lambda x: log22lin(float(3)) <= x and x < log22lin(float(3.5)),global_values))
    values_plus_3_5_to_4 = list(filter(lambda x: log22lin(float(3.5)) <= x and x < log22lin(float(4)),global_values))
    values_plus_4_to_4_5 = list(filter(lambda x: log22lin(float(4)) <= x and x < log22lin(float(4.5)),global_values))
    values_plus_4_5_to_5 = list(filter(lambda x: log22lin(float(4.5)) <= x and x < log22lin(float(5)),global_values))
    values_plus_5_to_5_5 = list(filter(lambda x: log22lin(float(5)) <= x and x < log22lin(float(5.5)),global_values))
    values_plus_5_5_to_6 = list(filter(lambda x: log22lin(float(5.5)) <= x and x < log22lin(float(6)),global_values))
    values_over_6 = list(filter(lambda x: x > log22lin(float(6)),global_values))

    value_counter_image_r = [len(values_lower_minus_10),
                    len(values_minus_10_to_9_5),
                    len(values_minus_9_5_to_9),
                    len(values_minus_9_to_8_5),
                    len(values_minus_8_5_to_8),
                    len(values_minus_8_to_7_5),
                    len(values_minus_7_5_to_7),
                    len(values_minus_7_to_6_5),
                    len(values_minus_6_5_to_6),
                    len(values_minus_6_to_5_5),
                    len(values_minus_5_5_to_5),
                    len(values_minus_5_to_4_5),
                    len(values_minus_4_5_to_4),
                    len(values_minus_4_to_3_5),
                    len(values_minus_3_5_to_3),
                    len(values_minus_3_to_2_5),
                    len(values_minus_2_5_to_2),
                    len(values_minus_2_to_1_5),
                    len(values_minus_1_5_to_1),
                    len(values_minus_1_to_0_5),
                    len(values_minus_0_5_to_0),
                    len(values_plus_0_to_0_5),
                    len(values_plus_0_5_to_1),
                    len(values_plus_1_to_1_5),
                    len(values_plus_1_5_to_2),
                    len(values_plus_2_to_2_5),
                    len(values_plus_2_5_to_3),
                    len(values_plus_3_to_3_5),
                    len(values_plus_3_5_to_4),
                    len(values_plus_4_to_4_5),
                    len(values_plus_4_5_to_5),
                    len(values_plus_5_to_5_5),
                    len(values_plus_5_5_to_6),
                    len(values_over_6)]




    global_values = axI_Scene_Linear_Line_g[:,0]

    values_lower_minus_10 = list(filter(lambda x: x < log22lin(float(-10)),global_values))
    values_minus_10_to_9_5 = list(filter(lambda x: log22lin(float(-10)) <= x and x < log22lin(float(-9.5)),global_values))
    values_minus_9_5_to_9 = list(filter(lambda x: log22lin(float(-9.5)) <= x and x < log22lin(float(-9)),global_values))
    values_minus_9_to_8_5 = list(filter(lambda x: log22lin(float(-9)) <= x and x < log22lin(float(-8.5)),global_values))
    values_minus_8_5_to_8 = list(filter(lambda x: log22lin(float(-8.5)) <= x and x < log22lin(float(-8)),global_values))
    values_minus_8_to_7_5 = list(filter(lambda x: log22lin(float(-8)) <= x and x < log22lin(float(-7.5)),global_values))
    values_minus_7_5_to_7 = list(filter(lambda x: log22lin(float(-7.5)) <= x and x < log22lin(float(-7)),global_values))
    values_minus_7_to_6_5 = list(filter(lambda x: log22lin(float(-7)) <= x and x < log22lin(float(-6.5)),global_values))
    values_minus_6_5_to_6 = list(filter(lambda x: log22lin(float(-6.5)) <= x and x < log22lin(float(-6)),global_values))
    values_minus_6_to_5_5 = list(filter(lambda x: log22lin(float(-6)) <= x and x < log22lin(float(-5.5)),global_values))
    values_minus_5_5_to_5 = list(filter(lambda x: log22lin(float(-5.5)) <= x and x < log22lin(float(-5)),global_values))
    values_minus_5_to_4_5 = list(filter(lambda x: log22lin(float(-5)) <= x and x < log22lin(float(-4.5)),global_values))
    values_minus_4_5_to_4 = list(filter(lambda x: log22lin(float(-4.5)) <= x and x < log22lin(float(-4)),global_values))
    values_minus_4_to_3_5 = list(filter(lambda x: log22lin(float(-4)) <= x and x < log22lin(float(-3.5)),global_values))
    values_minus_3_5_to_3 = list(filter(lambda x: log22lin(float(-3.5)) <= x and x < log22lin(float(-3)),global_values))
    values_minus_3_to_2_5 = list(filter(lambda x: log22lin(float(-3)) <= x and x < log22lin(float(-2.5)),global_values))
    values_minus_2_5_to_2 = list(filter(lambda x: log22lin(float(-2.5)) <= x and x < log22lin(float(-2)),global_values))
    values_minus_2_to_1_5 = list(filter(lambda x: log22lin(float(-2)) <= x and x < log22lin(float(-1.5)),global_values))
    values_minus_1_5_to_1 = list(filter(lambda x: log22lin(float(-1.5)) <= x and x < log22lin(float(-1)),global_values))
    values_minus_1_to_0_5 = list(filter(lambda x: log22lin(float(-1)) <= x and x < log22lin(float(-0.5)),global_values))
    values_minus_0_5_to_0 = list(filter(lambda x: log22lin(float(-0.5)) <= x and x < log22lin(float(0)),global_values))
    values_plus_0_to_0_5 = list(filter(lambda x: log22lin(float(0)) <= x and x < log22lin(float(0.5)),global_values))
    values_plus_0_5_to_1 = list(filter(lambda x: log22lin(float(0.5)) <= x and x < log22lin(float(1)),global_values))
    values_plus_1_to_1_5 = list(filter(lambda x: log22lin(float(1)) <= x and x < log22lin(float(1.5)),global_values))
    values_plus_1_5_to_2 = list(filter(lambda x: log22lin(float(1.5)) <= x and x < log22lin(float(2)),global_values))
    values_plus_2_to_2_5 = list(filter(lambda x: log22lin(float(2)) <= x and x < log22lin(float(2.5)),global_values))
    values_plus_2_5_to_3 = list(filter(lambda x: log22lin(float(2.5)) <= x and x < log22lin(float(3)),global_values))
    values_plus_3_to_3_5 = list(filter(lambda x: log22lin(float(3)) <= x and x < log22lin(float(3.5)),global_values))
    values_plus_3_5_to_4 = list(filter(lambda x: log22lin(float(3.5)) <= x and x < log22lin(float(4)),global_values))
    values_plus_4_to_4_5 = list(filter(lambda x: log22lin(float(4)) <= x and x < log22lin(float(4.5)),global_values))
    values_plus_4_5_to_5 = list(filter(lambda x: log22lin(float(4.5)) <= x and x < log22lin(float(5)),global_values))
    values_plus_5_to_5_5 = list(filter(lambda x: log22lin(float(5)) <= x and x < log22lin(float(5.5)),global_values))
    values_plus_5_5_to_6 = list(filter(lambda x: log22lin(float(5.5)) <= x and x < log22lin(float(6)),global_values))
    values_over_6 = list(filter(lambda x: x > log22lin(float(6)),global_values))

    value_counter_image_g = [len(values_lower_minus_10),
                    len(values_minus_10_to_9_5),
                    len(values_minus_9_5_to_9),
                    len(values_minus_9_to_8_5),
                    len(values_minus_8_5_to_8),
                    len(values_minus_8_to_7_5),
                    len(values_minus_7_5_to_7),
                    len(values_minus_7_to_6_5),
                    len(values_minus_6_5_to_6),
                    len(values_minus_6_to_5_5),
                    len(values_minus_5_5_to_5),
                    len(values_minus_5_to_4_5),
                    len(values_minus_4_5_to_4),
                    len(values_minus_4_to_3_5),
                    len(values_minus_3_5_to_3),
                    len(values_minus_3_to_2_5),
                    len(values_minus_2_5_to_2),
                    len(values_minus_2_to_1_5),
                    len(values_minus_1_5_to_1),
                    len(values_minus_1_to_0_5),
                    len(values_minus_0_5_to_0),
                    len(values_plus_0_to_0_5),
                    len(values_plus_0_5_to_1),
                    len(values_plus_1_to_1_5),
                    len(values_plus_1_5_to_2),
                    len(values_plus_2_to_2_5),
                    len(values_plus_2_5_to_3),
                    len(values_plus_3_to_3_5),
                    len(values_plus_3_5_to_4),
                    len(values_plus_4_to_4_5),
                    len(values_plus_4_5_to_5),
                    len(values_plus_5_to_5_5),
                    len(values_plus_5_5_to_6),
                    len(values_over_6)]




    global_values = axI_Scene_Linear_Line_b[:,0]

    values_lower_minus_10 = list(filter(lambda x: x < log22lin(float(-10)),global_values))
    values_minus_10_to_9_5 = list(filter(lambda x: log22lin(float(-10)) <= x and x < log22lin(float(-9.5)),global_values))
    values_minus_9_5_to_9 = list(filter(lambda x: log22lin(float(-9.5)) <= x and x < log22lin(float(-9)),global_values))
    values_minus_9_to_8_5 = list(filter(lambda x: log22lin(float(-9)) <= x and x < log22lin(float(-8.5)),global_values))
    values_minus_8_5_to_8 = list(filter(lambda x: log22lin(float(-8.5)) <= x and x < log22lin(float(-8)),global_values))
    values_minus_8_to_7_5 = list(filter(lambda x: log22lin(float(-8)) <= x and x < log22lin(float(-7.5)),global_values))
    values_minus_7_5_to_7 = list(filter(lambda x: log22lin(float(-7.5)) <= x and x < log22lin(float(-7)),global_values))
    values_minus_7_to_6_5 = list(filter(lambda x: log22lin(float(-7)) <= x and x < log22lin(float(-6.5)),global_values))
    values_minus_6_5_to_6 = list(filter(lambda x: log22lin(float(-6.5)) <= x and x < log22lin(float(-6)),global_values))
    values_minus_6_to_5_5 = list(filter(lambda x: log22lin(float(-6)) <= x and x < log22lin(float(-5.5)),global_values))
    values_minus_5_5_to_5 = list(filter(lambda x: log22lin(float(-5.5)) <= x and x < log22lin(float(-5)),global_values))
    values_minus_5_to_4_5 = list(filter(lambda x: log22lin(float(-5)) <= x and x < log22lin(float(-4.5)),global_values))
    values_minus_4_5_to_4 = list(filter(lambda x: log22lin(float(-4.5)) <= x and x < log22lin(float(-4)),global_values))
    values_minus_4_to_3_5 = list(filter(lambda x: log22lin(float(-4)) <= x and x < log22lin(float(-3.5)),global_values))
    values_minus_3_5_to_3 = list(filter(lambda x: log22lin(float(-3.5)) <= x and x < log22lin(float(-3)),global_values))
    values_minus_3_to_2_5 = list(filter(lambda x: log22lin(float(-3)) <= x and x < log22lin(float(-2.5)),global_values))
    values_minus_2_5_to_2 = list(filter(lambda x: log22lin(float(-2.5)) <= x and x < log22lin(float(-2)),global_values))
    values_minus_2_to_1_5 = list(filter(lambda x: log22lin(float(-2)) <= x and x < log22lin(float(-1.5)),global_values))
    values_minus_1_5_to_1 = list(filter(lambda x: log22lin(float(-1.5)) <= x and x < log22lin(float(-1)),global_values))
    values_minus_1_to_0_5 = list(filter(lambda x: log22lin(float(-1)) <= x and x < log22lin(float(-0.5)),global_values))
    values_minus_0_5_to_0 = list(filter(lambda x: log22lin(float(-0.5)) <= x and x < log22lin(float(0)),global_values))
    values_plus_0_to_0_5 = list(filter(lambda x: log22lin(float(0)) <= x and x < log22lin(float(0.5)),global_values))
    values_plus_0_5_to_1 = list(filter(lambda x: log22lin(float(0.5)) <= x and x < log22lin(float(1)),global_values))
    values_plus_1_to_1_5 = list(filter(lambda x: log22lin(float(1)) <= x and x < log22lin(float(1.5)),global_values))
    values_plus_1_5_to_2 = list(filter(lambda x: log22lin(float(1.5)) <= x and x < log22lin(float(2)),global_values))
    values_plus_2_to_2_5 = list(filter(lambda x: log22lin(float(2)) <= x and x < log22lin(float(2.5)),global_values))
    values_plus_2_5_to_3 = list(filter(lambda x: log22lin(float(2.5)) <= x and x < log22lin(float(3)),global_values))
    values_plus_3_to_3_5 = list(filter(lambda x: log22lin(float(3)) <= x and x < log22lin(float(3.5)),global_values))
    values_plus_3_5_to_4 = list(filter(lambda x: log22lin(float(3.5)) <= x and x < log22lin(float(4)),global_values))
    values_plus_4_to_4_5 = list(filter(lambda x: log22lin(float(4)) <= x and x < log22lin(float(4.5)),global_values))
    values_plus_4_5_to_5 = list(filter(lambda x: log22lin(float(4.5)) <= x and x < log22lin(float(5)),global_values))
    values_plus_5_to_5_5 = list(filter(lambda x: log22lin(float(5)) <= x and x < log22lin(float(5.5)),global_values))
    values_plus_5_5_to_6 = list(filter(lambda x: log22lin(float(5.5)) <= x and x < log22lin(float(6)),global_values))
    values_over_6 = list(filter(lambda x: x > log22lin(float(6)),global_values))

    value_counter_image_b = [len(values_lower_minus_10),
                    len(values_minus_10_to_9_5),
                    len(values_minus_9_5_to_9),
                    len(values_minus_9_to_8_5),
                    len(values_minus_8_5_to_8),
                    len(values_minus_8_to_7_5),
                    len(values_minus_7_5_to_7),
                    len(values_minus_7_to_6_5),
                    len(values_minus_6_5_to_6),
                    len(values_minus_6_to_5_5),
                    len(values_minus_5_5_to_5),
                    len(values_minus_5_to_4_5),
                    len(values_minus_4_5_to_4),
                    len(values_minus_4_to_3_5),
                    len(values_minus_3_5_to_3),
                    len(values_minus_3_to_2_5),
                    len(values_minus_2_5_to_2),
                    len(values_minus_2_to_1_5),
                    len(values_minus_1_5_to_1),
                    len(values_minus_1_to_0_5),
                    len(values_minus_0_5_to_0),
                    len(values_plus_0_to_0_5),
                    len(values_plus_0_5_to_1),
                    len(values_plus_1_to_1_5),
                    len(values_plus_1_5_to_2),
                    len(values_plus_2_to_2_5),
                    len(values_plus_2_5_to_3),
                    len(values_plus_3_to_3_5),
                    len(values_plus_3_5_to_4),
                    len(values_plus_4_to_4_5),
                    len(values_plus_4_5_to_5),
                    len(values_plus_5_to_5_5),
                    len(values_plus_5_5_to_6),
                    len(values_over_6)]


    bars = ('A', 'B', 'C', 'D', 'E',
        'F','G', 'H', 'I', 'J', 
        'K','L','M', 'N', 'O', 
        'P', 'Q','R','A', 'B', 
        'C', 'D', 'E','F','G', 
        'H', 'I', 'J', 'K','L',
        'M', 'N', 'O', 'P')

    y_pos = np.arange(len(bars))

    axV.bar(y_pos/2+0.15, value_counter_image_r, color="red", width=0.1, alpha=0.3, label="Scene Linear Light")
    axV.bar(y_pos/2+0.1+0.15, value_counter_image_g, color="green", width=0.1, alpha=0.3, label="Scene Linear Light")
    axV.bar(y_pos/2+0.2+0.15, value_counter_image_b, color="blue", width=0.1, alpha=0.3, label="Scene Linear Light")

    y_pos_between = np.linspace(-10,7,18)
    y_pos_neu = np.linspace(0,17,18)

    axV.set_xticks(y_pos_neu+0.5, y_pos_between)

    axV.set_xlim(-0.5,18.5)
    axV.set_yscale('log')
    axV.set_ylim(1,1000000)

    axV.legend()

    figR.savefig(f"/Users/justus/Library/CloudStorage/OneDrive-Persönlich/Masterarbeit/Analysis_Content_Encoding/ARRI/HDR_ANALYSIS_{i}.png")

In [ ]:
## Log C4 Encoding

for i in range(1,7):

    figR, (axI, axV) = plt.subplots(2, 1, figsize=(16, 16), constrained_layout=True, gridspec_kw={'height_ratios': [2, 1]})

    file = f"/Users/justus/Library/CloudStorage/OneDrive-Persönlich/Masterarbeit/Analysis_Content_Encoding/00_CONTENT/ARRI/HDR_{i}.tif"
    image_rgb = c.read_image(file)


    # axI_Scene_Linear = c.models.log_decoding_ALEXALogC(c.LUT3D.apply(Inverse_LUT_ARRI_4K_PQ_200,image_rgb))
    axI_Scene_Linear = Decoding_LogC4(image_rgb)

    axI_Scene_Linear_Line_r = axI_Scene_Linear[:,:,0].reshape(-1, 1) * 100
    axI_Scene_Linear_Line_g = axI_Scene_Linear[:,:,1].reshape(-1, 1) * 100
    axI_Scene_Linear_Line_b = axI_Scene_Linear[:,:,2].reshape(-1, 1) * 100

    value_counter_image_r = np.zeros((34))
    value_counter_image_g = np.zeros((34))
    value_counter_image_b = np.zeros((34))

    axI.imshow(c.LUT3D.apply(LUT_ARRI_C4_to_709_D65,image_rgb))
    # axI.text(5,1050,f"HDR-SDR-Transform with 5-NBCU_PQ2SDR_DL_v1.cube", color='red', fontsize=10, fontstyle='normal')


    axV.set_ylabel('Amount of Code Values Used [CV]')

    def full2legal(x):
        return (219*(x)+16)*np.power(2,(10-8))

    def legal2full(x):
        return (((x/np.power(2,(10-8)))-16)/219)

    def lin2log2(x):
        return np.log2(x)

    def log22lin(x):
        return np.power(2,x) * 100

    #x1 = np.linspace(0.000000000000001,1,1000000)

    axV.vlines(0.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
    axV.vlines(1.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
    axV.vlines(2.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
    axV.vlines(3.5,0,10000000,color='black', linestyle="solid", alpha=0.5,linewidth=2)
    axV.vlines(4.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
    axV.vlines(5.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
    axV.vlines(6.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
    axV.vlines(7.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
    axV.vlines(8.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
    axV.vlines(9.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
    axV.vlines(10.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
    axV.vlines(11.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
    axV.vlines(12.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
    axV.vlines(13.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
    axV.vlines(14.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
    axV.vlines(15.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
    axV.vlines(16.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
    axV.vlines(17.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)



    axV.vlines(1,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
    axV.vlines(2,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
    axV.vlines(3,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
    axV.vlines(4,0,10000000,color='black', linestyle="dashed", alpha=0.4)
    axV.vlines(5,0,10000000,color='black', linestyle="dashed", alpha=0.4)
    axV.vlines(6,0,10000000,color='black', linestyle="dashed", alpha=0.4)
    axV.vlines(7,0,10000000,color='black', linestyle="dashed", alpha=0.4)
    axV.vlines(8,0,10000000,color='black', linestyle="dashed", alpha=0.4)
    axV.vlines(9,0,10000000,color='black', linestyle="dashed", alpha=0.4)
    axV.vlines(10,0,10000000,color='black', linestyle="dashed", alpha=0.4)
    axV.vlines(11,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
    axV.vlines(12,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
    axV.vlines(13,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
    axV.vlines(14,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
    axV.vlines(15,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
    axV.vlines(16,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
    axV.vlines(17,0,10000000,color='gray', linestyle="dashed", alpha=0.4)



    global_values = axI_Scene_Linear_Line_r[:,0]

    values_lower_minus_10 = list(filter(lambda x: x < log22lin(float(-10)),global_values))
    values_minus_10_to_9_5 = list(filter(lambda x: log22lin(float(-10)) <= x and x < log22lin(float(-9.5)),global_values))
    values_minus_9_5_to_9 = list(filter(lambda x: log22lin(float(-9.5)) <= x and x < log22lin(float(-9)),global_values))
    values_minus_9_to_8_5 = list(filter(lambda x: log22lin(float(-9)) <= x and x < log22lin(float(-8.5)),global_values))
    values_minus_8_5_to_8 = list(filter(lambda x: log22lin(float(-8.5)) <= x and x < log22lin(float(-8)),global_values))
    values_minus_8_to_7_5 = list(filter(lambda x: log22lin(float(-8)) <= x and x < log22lin(float(-7.5)),global_values))
    values_minus_7_5_to_7 = list(filter(lambda x: log22lin(float(-7.5)) <= x and x < log22lin(float(-7)),global_values))
    values_minus_7_to_6_5 = list(filter(lambda x: log22lin(float(-7)) <= x and x < log22lin(float(-6.5)),global_values))
    values_minus_6_5_to_6 = list(filter(lambda x: log22lin(float(-6.5)) <= x and x < log22lin(float(-6)),global_values))
    values_minus_6_to_5_5 = list(filter(lambda x: log22lin(float(-6)) <= x and x < log22lin(float(-5.5)),global_values))
    values_minus_5_5_to_5 = list(filter(lambda x: log22lin(float(-5.5)) <= x and x < log22lin(float(-5)),global_values))
    values_minus_5_to_4_5 = list(filter(lambda x: log22lin(float(-5)) <= x and x < log22lin(float(-4.5)),global_values))
    values_minus_4_5_to_4 = list(filter(lambda x: log22lin(float(-4.5)) <= x and x < log22lin(float(-4)),global_values))
    values_minus_4_to_3_5 = list(filter(lambda x: log22lin(float(-4)) <= x and x < log22lin(float(-3.5)),global_values))
    values_minus_3_5_to_3 = list(filter(lambda x: log22lin(float(-3.5)) <= x and x < log22lin(float(-3)),global_values))
    values_minus_3_to_2_5 = list(filter(lambda x: log22lin(float(-3)) <= x and x < log22lin(float(-2.5)),global_values))
    values_minus_2_5_to_2 = list(filter(lambda x: log22lin(float(-2.5)) <= x and x < log22lin(float(-2)),global_values))
    values_minus_2_to_1_5 = list(filter(lambda x: log22lin(float(-2)) <= x and x < log22lin(float(-1.5)),global_values))
    values_minus_1_5_to_1 = list(filter(lambda x: log22lin(float(-1.5)) <= x and x < log22lin(float(-1)),global_values))
    values_minus_1_to_0_5 = list(filter(lambda x: log22lin(float(-1)) <= x and x < log22lin(float(-0.5)),global_values))
    values_minus_0_5_to_0 = list(filter(lambda x: log22lin(float(-0.5)) <= x and x < log22lin(float(0)),global_values))
    values_plus_0_to_0_5 = list(filter(lambda x: log22lin(float(0)) <= x and x < log22lin(float(0.5)),global_values))
    values_plus_0_5_to_1 = list(filter(lambda x: log22lin(float(0.5)) <= x and x < log22lin(float(1)),global_values))
    values_plus_1_to_1_5 = list(filter(lambda x: log22lin(float(1)) <= x and x < log22lin(float(1.5)),global_values))
    values_plus_1_5_to_2 = list(filter(lambda x: log22lin(float(1.5)) <= x and x < log22lin(float(2)),global_values))
    values_plus_2_to_2_5 = list(filter(lambda x: log22lin(float(2)) <= x and x < log22lin(float(2.5)),global_values))
    values_plus_2_5_to_3 = list(filter(lambda x: log22lin(float(2.5)) <= x and x < log22lin(float(3)),global_values))
    values_plus_3_to_3_5 = list(filter(lambda x: log22lin(float(3)) <= x and x < log22lin(float(3.5)),global_values))
    values_plus_3_5_to_4 = list(filter(lambda x: log22lin(float(3.5)) <= x and x < log22lin(float(4)),global_values))
    values_plus_4_to_4_5 = list(filter(lambda x: log22lin(float(4)) <= x and x < log22lin(float(4.5)),global_values))
    values_plus_4_5_to_5 = list(filter(lambda x: log22lin(float(4.5)) <= x and x < log22lin(float(5)),global_values))
    values_plus_5_to_5_5 = list(filter(lambda x: log22lin(float(5)) <= x and x < log22lin(float(5.5)),global_values))
    values_plus_5_5_to_6 = list(filter(lambda x: log22lin(float(5.5)) <= x and x < log22lin(float(6)),global_values))
    values_over_6 = list(filter(lambda x: x > log22lin(float(6)),global_values))

    value_counter_image_r = [len(values_lower_minus_10),
                    len(values_minus_10_to_9_5),
                    len(values_minus_9_5_to_9),
                    len(values_minus_9_to_8_5),
                    len(values_minus_8_5_to_8),
                    len(values_minus_8_to_7_5),
                    len(values_minus_7_5_to_7),
                    len(values_minus_7_to_6_5),
                    len(values_minus_6_5_to_6),
                    len(values_minus_6_to_5_5),
                    len(values_minus_5_5_to_5),
                    len(values_minus_5_to_4_5),
                    len(values_minus_4_5_to_4),
                    len(values_minus_4_to_3_5),
                    len(values_minus_3_5_to_3),
                    len(values_minus_3_to_2_5),
                    len(values_minus_2_5_to_2),
                    len(values_minus_2_to_1_5),
                    len(values_minus_1_5_to_1),
                    len(values_minus_1_to_0_5),
                    len(values_minus_0_5_to_0),
                    len(values_plus_0_to_0_5),
                    len(values_plus_0_5_to_1),
                    len(values_plus_1_to_1_5),
                    len(values_plus_1_5_to_2),
                    len(values_plus_2_to_2_5),
                    len(values_plus_2_5_to_3),
                    len(values_plus_3_to_3_5),
                    len(values_plus_3_5_to_4),
                    len(values_plus_4_to_4_5),
                    len(values_plus_4_5_to_5),
                    len(values_plus_5_to_5_5),
                    len(values_plus_5_5_to_6),
                    len(values_over_6)]




    global_values = axI_Scene_Linear_Line_g[:,0]

    values_lower_minus_10 = list(filter(lambda x: x < log22lin(float(-10)),global_values))
    values_minus_10_to_9_5 = list(filter(lambda x: log22lin(float(-10)) <= x and x < log22lin(float(-9.5)),global_values))
    values_minus_9_5_to_9 = list(filter(lambda x: log22lin(float(-9.5)) <= x and x < log22lin(float(-9)),global_values))
    values_minus_9_to_8_5 = list(filter(lambda x: log22lin(float(-9)) <= x and x < log22lin(float(-8.5)),global_values))
    values_minus_8_5_to_8 = list(filter(lambda x: log22lin(float(-8.5)) <= x and x < log22lin(float(-8)),global_values))
    values_minus_8_to_7_5 = list(filter(lambda x: log22lin(float(-8)) <= x and x < log22lin(float(-7.5)),global_values))
    values_minus_7_5_to_7 = list(filter(lambda x: log22lin(float(-7.5)) <= x and x < log22lin(float(-7)),global_values))
    values_minus_7_to_6_5 = list(filter(lambda x: log22lin(float(-7)) <= x and x < log22lin(float(-6.5)),global_values))
    values_minus_6_5_to_6 = list(filter(lambda x: log22lin(float(-6.5)) <= x and x < log22lin(float(-6)),global_values))
    values_minus_6_to_5_5 = list(filter(lambda x: log22lin(float(-6)) <= x and x < log22lin(float(-5.5)),global_values))
    values_minus_5_5_to_5 = list(filter(lambda x: log22lin(float(-5.5)) <= x and x < log22lin(float(-5)),global_values))
    values_minus_5_to_4_5 = list(filter(lambda x: log22lin(float(-5)) <= x and x < log22lin(float(-4.5)),global_values))
    values_minus_4_5_to_4 = list(filter(lambda x: log22lin(float(-4.5)) <= x and x < log22lin(float(-4)),global_values))
    values_minus_4_to_3_5 = list(filter(lambda x: log22lin(float(-4)) <= x and x < log22lin(float(-3.5)),global_values))
    values_minus_3_5_to_3 = list(filter(lambda x: log22lin(float(-3.5)) <= x and x < log22lin(float(-3)),global_values))
    values_minus_3_to_2_5 = list(filter(lambda x: log22lin(float(-3)) <= x and x < log22lin(float(-2.5)),global_values))
    values_minus_2_5_to_2 = list(filter(lambda x: log22lin(float(-2.5)) <= x and x < log22lin(float(-2)),global_values))
    values_minus_2_to_1_5 = list(filter(lambda x: log22lin(float(-2)) <= x and x < log22lin(float(-1.5)),global_values))
    values_minus_1_5_to_1 = list(filter(lambda x: log22lin(float(-1.5)) <= x and x < log22lin(float(-1)),global_values))
    values_minus_1_to_0_5 = list(filter(lambda x: log22lin(float(-1)) <= x and x < log22lin(float(-0.5)),global_values))
    values_minus_0_5_to_0 = list(filter(lambda x: log22lin(float(-0.5)) <= x and x < log22lin(float(0)),global_values))
    values_plus_0_to_0_5 = list(filter(lambda x: log22lin(float(0)) <= x and x < log22lin(float(0.5)),global_values))
    values_plus_0_5_to_1 = list(filter(lambda x: log22lin(float(0.5)) <= x and x < log22lin(float(1)),global_values))
    values_plus_1_to_1_5 = list(filter(lambda x: log22lin(float(1)) <= x and x < log22lin(float(1.5)),global_values))
    values_plus_1_5_to_2 = list(filter(lambda x: log22lin(float(1.5)) <= x and x < log22lin(float(2)),global_values))
    values_plus_2_to_2_5 = list(filter(lambda x: log22lin(float(2)) <= x and x < log22lin(float(2.5)),global_values))
    values_plus_2_5_to_3 = list(filter(lambda x: log22lin(float(2.5)) <= x and x < log22lin(float(3)),global_values))
    values_plus_3_to_3_5 = list(filter(lambda x: log22lin(float(3)) <= x and x < log22lin(float(3.5)),global_values))
    values_plus_3_5_to_4 = list(filter(lambda x: log22lin(float(3.5)) <= x and x < log22lin(float(4)),global_values))
    values_plus_4_to_4_5 = list(filter(lambda x: log22lin(float(4)) <= x and x < log22lin(float(4.5)),global_values))
    values_plus_4_5_to_5 = list(filter(lambda x: log22lin(float(4.5)) <= x and x < log22lin(float(5)),global_values))
    values_plus_5_to_5_5 = list(filter(lambda x: log22lin(float(5)) <= x and x < log22lin(float(5.5)),global_values))
    values_plus_5_5_to_6 = list(filter(lambda x: log22lin(float(5.5)) <= x and x < log22lin(float(6)),global_values))
    values_over_6 = list(filter(lambda x: x > log22lin(float(6)),global_values))

    value_counter_image_g = [len(values_lower_minus_10),
                    len(values_minus_10_to_9_5),
                    len(values_minus_9_5_to_9),
                    len(values_minus_9_to_8_5),
                    len(values_minus_8_5_to_8),
                    len(values_minus_8_to_7_5),
                    len(values_minus_7_5_to_7),
                    len(values_minus_7_to_6_5),
                    len(values_minus_6_5_to_6),
                    len(values_minus_6_to_5_5),
                    len(values_minus_5_5_to_5),
                    len(values_minus_5_to_4_5),
                    len(values_minus_4_5_to_4),
                    len(values_minus_4_to_3_5),
                    len(values_minus_3_5_to_3),
                    len(values_minus_3_to_2_5),
                    len(values_minus_2_5_to_2),
                    len(values_minus_2_to_1_5),
                    len(values_minus_1_5_to_1),
                    len(values_minus_1_to_0_5),
                    len(values_minus_0_5_to_0),
                    len(values_plus_0_to_0_5),
                    len(values_plus_0_5_to_1),
                    len(values_plus_1_to_1_5),
                    len(values_plus_1_5_to_2),
                    len(values_plus_2_to_2_5),
                    len(values_plus_2_5_to_3),
                    len(values_plus_3_to_3_5),
                    len(values_plus_3_5_to_4),
                    len(values_plus_4_to_4_5),
                    len(values_plus_4_5_to_5),
                    len(values_plus_5_to_5_5),
                    len(values_plus_5_5_to_6),
                    len(values_over_6)]




    global_values = axI_Scene_Linear_Line_b[:,0]

    values_lower_minus_10 = list(filter(lambda x: x < log22lin(float(-10)),global_values))
    values_minus_10_to_9_5 = list(filter(lambda x: log22lin(float(-10)) <= x and x < log22lin(float(-9.5)),global_values))
    values_minus_9_5_to_9 = list(filter(lambda x: log22lin(float(-9.5)) <= x and x < log22lin(float(-9)),global_values))
    values_minus_9_to_8_5 = list(filter(lambda x: log22lin(float(-9)) <= x and x < log22lin(float(-8.5)),global_values))
    values_minus_8_5_to_8 = list(filter(lambda x: log22lin(float(-8.5)) <= x and x < log22lin(float(-8)),global_values))
    values_minus_8_to_7_5 = list(filter(lambda x: log22lin(float(-8)) <= x and x < log22lin(float(-7.5)),global_values))
    values_minus_7_5_to_7 = list(filter(lambda x: log22lin(float(-7.5)) <= x and x < log22lin(float(-7)),global_values))
    values_minus_7_to_6_5 = list(filter(lambda x: log22lin(float(-7)) <= x and x < log22lin(float(-6.5)),global_values))
    values_minus_6_5_to_6 = list(filter(lambda x: log22lin(float(-6.5)) <= x and x < log22lin(float(-6)),global_values))
    values_minus_6_to_5_5 = list(filter(lambda x: log22lin(float(-6)) <= x and x < log22lin(float(-5.5)),global_values))
    values_minus_5_5_to_5 = list(filter(lambda x: log22lin(float(-5.5)) <= x and x < log22lin(float(-5)),global_values))
    values_minus_5_to_4_5 = list(filter(lambda x: log22lin(float(-5)) <= x and x < log22lin(float(-4.5)),global_values))
    values_minus_4_5_to_4 = list(filter(lambda x: log22lin(float(-4.5)) <= x and x < log22lin(float(-4)),global_values))
    values_minus_4_to_3_5 = list(filter(lambda x: log22lin(float(-4)) <= x and x < log22lin(float(-3.5)),global_values))
    values_minus_3_5_to_3 = list(filter(lambda x: log22lin(float(-3.5)) <= x and x < log22lin(float(-3)),global_values))
    values_minus_3_to_2_5 = list(filter(lambda x: log22lin(float(-3)) <= x and x < log22lin(float(-2.5)),global_values))
    values_minus_2_5_to_2 = list(filter(lambda x: log22lin(float(-2.5)) <= x and x < log22lin(float(-2)),global_values))
    values_minus_2_to_1_5 = list(filter(lambda x: log22lin(float(-2)) <= x and x < log22lin(float(-1.5)),global_values))
    values_minus_1_5_to_1 = list(filter(lambda x: log22lin(float(-1.5)) <= x and x < log22lin(float(-1)),global_values))
    values_minus_1_to_0_5 = list(filter(lambda x: log22lin(float(-1)) <= x and x < log22lin(float(-0.5)),global_values))
    values_minus_0_5_to_0 = list(filter(lambda x: log22lin(float(-0.5)) <= x and x < log22lin(float(0)),global_values))
    values_plus_0_to_0_5 = list(filter(lambda x: log22lin(float(0)) <= x and x < log22lin(float(0.5)),global_values))
    values_plus_0_5_to_1 = list(filter(lambda x: log22lin(float(0.5)) <= x and x < log22lin(float(1)),global_values))
    values_plus_1_to_1_5 = list(filter(lambda x: log22lin(float(1)) <= x and x < log22lin(float(1.5)),global_values))
    values_plus_1_5_to_2 = list(filter(lambda x: log22lin(float(1.5)) <= x and x < log22lin(float(2)),global_values))
    values_plus_2_to_2_5 = list(filter(lambda x: log22lin(float(2)) <= x and x < log22lin(float(2.5)),global_values))
    values_plus_2_5_to_3 = list(filter(lambda x: log22lin(float(2.5)) <= x and x < log22lin(float(3)),global_values))
    values_plus_3_to_3_5 = list(filter(lambda x: log22lin(float(3)) <= x and x < log22lin(float(3.5)),global_values))
    values_plus_3_5_to_4 = list(filter(lambda x: log22lin(float(3.5)) <= x and x < log22lin(float(4)),global_values))
    values_plus_4_to_4_5 = list(filter(lambda x: log22lin(float(4)) <= x and x < log22lin(float(4.5)),global_values))
    values_plus_4_5_to_5 = list(filter(lambda x: log22lin(float(4.5)) <= x and x < log22lin(float(5)),global_values))
    values_plus_5_to_5_5 = list(filter(lambda x: log22lin(float(5)) <= x and x < log22lin(float(5.5)),global_values))
    values_plus_5_5_to_6 = list(filter(lambda x: log22lin(float(5.5)) <= x and x < log22lin(float(6)),global_values))
    values_over_6 = list(filter(lambda x: x > log22lin(float(6)),global_values))

    value_counter_image_b = [len(values_lower_minus_10),
                    len(values_minus_10_to_9_5),
                    len(values_minus_9_5_to_9),
                    len(values_minus_9_to_8_5),
                    len(values_minus_8_5_to_8),
                    len(values_minus_8_to_7_5),
                    len(values_minus_7_5_to_7),
                    len(values_minus_7_to_6_5),
                    len(values_minus_6_5_to_6),
                    len(values_minus_6_to_5_5),
                    len(values_minus_5_5_to_5),
                    len(values_minus_5_to_4_5),
                    len(values_minus_4_5_to_4),
                    len(values_minus_4_to_3_5),
                    len(values_minus_3_5_to_3),
                    len(values_minus_3_to_2_5),
                    len(values_minus_2_5_to_2),
                    len(values_minus_2_to_1_5),
                    len(values_minus_1_5_to_1),
                    len(values_minus_1_to_0_5),
                    len(values_minus_0_5_to_0),
                    len(values_plus_0_to_0_5),
                    len(values_plus_0_5_to_1),
                    len(values_plus_1_to_1_5),
                    len(values_plus_1_5_to_2),
                    len(values_plus_2_to_2_5),
                    len(values_plus_2_5_to_3),
                    len(values_plus_3_to_3_5),
                    len(values_plus_3_5_to_4),
                    len(values_plus_4_to_4_5),
                    len(values_plus_4_5_to_5),
                    len(values_plus_5_to_5_5),
                    len(values_plus_5_5_to_6),
                    len(values_over_6)]


    bars = ('A', 'B', 'C', 'D', 'E',
        'F','G', 'H', 'I', 'J', 
        'K','L','M', 'N', 'O', 
        'P', 'Q','R','A', 'B', 
        'C', 'D', 'E','F','G', 
        'H', 'I', 'J', 'K','L',
        'M', 'N', 'O', 'P')

    y_pos = np.arange(len(bars))

    axV.bar(y_pos/2+0.15, value_counter_image_r, color="red", width=0.1, alpha=0.3, label="Scene Linear Light Red Channel")
    axV.bar(y_pos/2+0.1+0.15, value_counter_image_g, color="green", width=0.1, alpha=0.3, label="Scene Linear Light Green Channel")
    axV.bar(y_pos/2+0.2+0.15, value_counter_image_b, color="blue", width=0.1, alpha=0.3, label="Scene Linear Light Blue Channel")

    y_pos_between = np.linspace(-10,7,18)
    y_pos_neu = np.linspace(0,17,18)

    axV.set_xticks(y_pos_neu+0.5, y_pos_between)

    axV.set_xlim(-0.5,18.5)d
    axV.set_yscale('log')
    axV.set_ylim(1,10000000)

    axV.legend()

    figR.savefig(f"/Users/justus/Library/CloudStorage/OneDrive-Persönlich/Masterarbeit/Analysis_Content_Encoding/ARRI/HDR_ANALYSIS_HDR_{i}.png")

In [ ]:
from numba.typed import List
from numba import njit
import imageio as iio

In [ ]:
log22lin(float(5.5))

In [ ]:
#@njit
def values_lower_minus_10(array):
    index = (array[:] < 0.0009765625)
    return array[index]

#@njit
def values_minus_10_to_9_5(array):
    index = (0.0009765625 <= array[:,]) & (array[:] < 0.0013810679320049757)
    return array[index]

#@njit
def values_minus_9_5_to_9(array):
    index = (0.0013810679320049757 <= array[:]) & (array[:] < 0.001953125)
    return array[index]

#@njit
def values_minus_9_to_8_5(array):
    index = (0.001953125 <= array[:]) & (array[:] < 0.0027621358640099515)
    return array[index]

#@njit
def values_minus_8_5_to_8(array):
    index = (0.0027621358640099515 <= array[:]) & (array[:] < 0.00390625)
    return array[index]

#@njit
def values_minus_8_to_7_5(array):
    index = (0.00390625 <= array[:]) & (array[:] < 0.0055242717280199029)
    return array[index]

#@njit
def values_minus_7_5_to_7(array):
    index = (0.0055242717280199029 <= array[:]) & (array[:] < 0.0078125)
    return array[index]

#@njit
def values_minus_7_to_6_5(array):
    index = (0.0078125 <= array[:]) & (array[:] < 0.011048543456039806)
    return array[index]

#@njit
def values_minus_6_5_to_6(array):
    index = (0.011048543456039806 <= array[:]) & (array[:] < 0.015625)
    return array[index]

#@njit
def values_minus_6_to_5_5(array):
    index = (0.015625 <= array[:]) & (array[:] < 0.022097086912079612)
    return array[index]

#@njit
def values_minus_5_5_to_5(array):
    index = (0.022097086912079612 <= array[:]) & (array[:] <= 0.03125)
    return array[index]

#@njit
def values_minus_5_to_4_5(array):
    index = (0.03125 <= array[:]) & (array[:] < 0.044194173824159223)
    return array[index]

#@njit
def values_minus_4_5_to_4(array):
    index = (0.044194173824159223 <= array[:]) & (array[:] <= 0.0625)
    return array[index]

#@njit
def values_minus_4_to_3_5(array):
    index = (0.0625 <= array[:]) & (array[:] < 0.088388347648318447)
    return array[index]

#@njit
def values_minus_3_5_to_3(array):
    index = (0.088388347648318447 <= array[:]) & (array[:] <= 0.125)
    return array[index]

#@njit
def values_minus_3_to_2_5(array):
    index = (0.125 <= array[:]) & (array[:] < 0.17677669529663689)
    return array[index]

#@njit
def values_minus_2_5_to_2(array):
    index = (0.17677669529663689 <= array[:]) & (array[:] <= 0.25)
    return array[index]

#@njit
def values_minus_2_to_1_5(array):
    index = (0.25 <= array[:]) & (array[:] < 0.35355339059327379)
    return array[index]

#@njit
def values_minus_1_5_to_1(array):
    index = (0.35355339059327379 <= array[:]) & (array[:] <= 0.5)
    return array[index]

#@njit
def values_minus_1_to_0_5(array):
    index = (0.5 <= array[:]) & (array[:] < 0.70710678118654757)
    return array[index]

#@njit
def values_minus_0_5_to_0(array):
    index = (0.70710678118654757 <= array[:]) & (array[:] <= 1)
    return array[index]

#@njit
def values_plus_0_to_0_5(array):
    index = (1 <= array[:]) & (array[:] < 1.4142135623730951)
    return array[index]

#@njit
def values_plus_0_5_to_1(array):
    index = (1.4142135623730951 <= array[:]) & (array[:] <= 2)
    return array[index]

#@njit
def values_plus_1_to_1_5(array):
    index = (2 <= array[:]) & (array[:] < 2.8284271247461903)
    return array[index]

#@njit
def values_plus_1_5_to_2(array):
    index = (2.8284271247461903 <= array[:]) & (array[:] <= 4)
    return array[index]

#@njit
def values_plus_2_to_2_5(array):
    index = (4 <= array[:]) & (array[:] < 5.6568542494923806)
    return array[index]

#@njit
def values_plus_2_5_to_3(array):
    index = (5.6568542494923806 <= array[:]) & (array[:] <= 8)
    return array[index]

#@njit
def values_plus_3_to_3_5(array):
    index = (8 <= array[:]) & (array[:] < 11.313708498984761)
    return array[index]

#@njit
def values_plus_3_5_to_4(array):
    index = (11.313708498984761 <= array[:]) & (array[:] <= 16)
    return array[index]

#@njit
def values_plus_4_to_4_5(array):
    index = (16 <= array[:]) & (array[:] < 22.627416997969522)
    return array[index]

#@njit
def values_plus_4_5_to_5(array):
    index = (22.627416997969522 <= array[:]) & (array[:] <= 32)
    return array[index]

#@njit
def values_plus_5_to_5_5(array):
    index = (32 <= array[:]) & (array[:] < 45.254833995939045)
    return array[index]

#@njit
def values_plus_5_5_to_6(array):
    index = (45.254833995939045 <= array[:]) & (array[:] < 64)
    return array[index]

#@njit
def values_over_6(array):
    index = (64 <= array[:])
    return array[index]

In [1]:
#@njit
def values_lower_minus_11(array):
    index = (0.005 <= array[:,]) & (array[:] < 0.006103515625)
    return array[index]

#@njit
def values_minus_11_to_10_5(array):
    index = (0.006103515625 <= array[:,]) & (array[:] < 0.0091552734375)
    return array[index]

#@njit
def values_minus_10_5_to_10(array):
    index = (0.0091552734375 <= array[:,]) & (array[:] < 0.01220703125)
    return array[index]

#@njit
def values_minus_10_to_9_5(array):
    index = (0.01220703125 <= array[:,]) & (array[:] < 0.018310546875)
    return array[index]

#@njit
def values_minus_9_5_to_9(array):
    index = (0.018310546875 <= array[:]) & (array[:] < 0.0244140625)
    return array[index]

#@njit
def values_minus_9_to_8_5(array):
    index = (0.0244140625 <= array[:]) & (array[:] < 0.03662109375)
    return array[index]

#@njit
def values_minus_8_5_to_8(array):
    index = (0.03662109375 <= array[:]) & (array[:] < 0.048828125)
    return array[index]

#@njit
def values_minus_8_to_7_5(array):
    index = (0.048828125 <= array[:]) & (array[:] < 0.0732421875)
    return array[index]

#@njit
def values_minus_7_5_to_7(array):
    index = (0.0732421875 <= array[:]) & (array[:] < 0.09765625)
    return array[index]

#@njit
def values_minus_7_to_6_5(array):
    index = (0.09765625 <= array[:]) & (array[:] < 0.146484375)
    return array[index]

#@njit
def values_minus_6_5_to_6(array):
    index = (0.146484375 <= array[:]) & (array[:] < 0.1953125)
    return array[index]

#@njit
def values_minus_6_to_5_5(array):
    index = (0.1953125 <= array[:]) & (array[:] < 0.29296875)
    return array[index]

#@njit
def values_minus_5_5_to_5(array):
    index = (0.29296875 <= array[:]) & (array[:] < 0.390625)
    return array[index]

#@njit
def values_minus_5_to_4_5(array):
    index = (0.390625 <= array[:]) & (array[:] < 0.5859375)
    return array[index]

#@njit
def values_minus_4_5_to_4(array):
    index = (0.5859375 <= array[:]) & (array[:] < 0.78125)
    return array[index]

#@njit
def values_minus_4_to_3_5(array):
    index = (0.78125 <= array[:]) & (array[:] < 1.171875)
    return array[index]

#@njit
def values_minus_3_5_to_3(array):
    index = (1.171875 <= array[:]) & (array[:] < 1.5625)
    return array[index]

#@njit
def values_minus_3_to_2_5(array):
    index = (1.5625 <= array[:]) & (array[:] < 2.34375)
    return array[index]

#@njit
def values_minus_2_5_to_2(array):
    index = (2.34375 <= array[:]) & (array[:] < 3.125)
    return array[index]

#@njit
def values_minus_2_to_1_5(array):
    index = (3.125 <= array[:]) & (array[:] < 4.6875)
    return array[index]

#@njit
def values_minus_1_5_to_1(array):
    index = (4.6875 <= array[:]) & (array[:] < 6.25)
    return array[index]

#@njit
def values_minus_1_to_0_5(array):
    index = (6.25 <= array[:]) & (array[:] < 9.375)
    return array[index]

#@njit
def values_minus_0_5_to_0(array):
    index = (9.375 <= array[:]) & (array[:] < 12.5)
    return array[index]

#@njit
def values_plus_0_to_0_5(array):
    index = (12.5 <= array[:]) & (array[:] < 18.75)
    return array[index]

#@njit
def values_plus_0_5_to_1(array):
    index = (18.75 <= array[:]) & (array[:] < 25)
    return array[index]

#@njit
def values_plus_1_to_1_5(array):
    index = (25 <= array[:]) & (array[:] < 37.5)
    return array[index]

#@njit
def values_plus_1_5_to_2(array):
    index = (37.5 <= array[:]) & (array[:] < 50)
    return array[index]

#@njit
def values_plus_2_to_2_5(array):
    index = (50 <= array[:]) & (array[:] < 75)
    return array[index]

#@njit
def values_plus_2_5_to_3(array):
    index = (75 <= array[:]) & (array[:] < 100)
    return array[index]

#@njit
def values_plus_3_to_3_5(array):
    index = (100 <= array[:]) & (array[:] < 150)
    return array[index]

#@njit
def values_plus_3_5_to_4(array):
    index = (150 <= array[:]) & (array[:] < 200)
    return array[index]

#@njit
def values_plus_4_to_4_5(array):
    index = (200 <= array[:]) & (array[:] < 300)
    return array[index]

#@njit
def values_plus_4_5_to_5(array):
    index = (300 <= array[:]) & (array[:] < 400)
    return array[index]

#@njit
def values_plus_5_to_5_5(array):
    index = (400 <= array[:]) & (array[:] < 600)
    return array[index]

#@njit
def values_plus_5_5_to_6(array):
    index = (600 <= array[:]) & (array[:] < 800)
    return array[index]

#@njit
def values_over_6(array):
    index = (800 <= array[:])
    return array[index]

In [2]:
## Log C4 Encoding


def split_array_to_stops(input):

    value_counter_image = np.zeros((36))

    global_values = input
    
    value_counter_image = [len(values_lower_minus_11(global_values)), 
                           len(values_minus_11_to_10_5(global_values)),
                           len(values_minus_10_5_to_10(global_values)),
                           len(values_minus_10_to_9_5(global_values)),
                           len(values_minus_9_5_to_9(global_values)),
                           len(values_minus_9_to_8_5(global_values)),
                           len(values_minus_8_5_to_8(global_values)),
                           len(values_minus_8_to_7_5(global_values)),
                           len(values_minus_7_5_to_7(global_values)),
                           len(values_minus_7_to_6_5(global_values)),
                           len(values_minus_6_5_to_6(global_values)),
                           len(values_minus_6_to_5_5(global_values)),
                           len(values_minus_5_5_to_5(global_values)),
                           len(values_minus_5_to_4_5(global_values)),
                           len(values_minus_4_5_to_4(global_values)),
                           len(values_minus_4_to_3_5(global_values)),
                           len(values_minus_3_5_to_3(global_values)),
                           len(values_minus_3_to_2_5(global_values)),
                           len(values_minus_2_5_to_2(global_values)),
                           len(values_minus_2_to_1_5(global_values)),
                           len(values_minus_1_5_to_1(global_values)),
                           len(values_minus_1_to_0_5(global_values)),
                           len(values_minus_0_5_to_0(global_values)),
                           len(values_plus_0_to_0_5(global_values)),
                           len(values_plus_0_5_to_1(global_values)),
                           len(values_plus_1_to_1_5(global_values)),
                           len(values_plus_1_5_to_2(global_values)),
                           len(values_plus_2_to_2_5(global_values)),
                           len(values_plus_2_5_to_3(global_values)),
                           len(values_plus_3_to_3_5(global_values)),
                           len(values_plus_3_5_to_4(global_values)),
                           len(values_plus_4_to_4_5(global_values)),
                           len(values_plus_4_5_to_5(global_values)),
                           len(values_plus_5_to_5_5(global_values)),
                           len(values_plus_5_5_to_6(global_values)),
                           len(values_over_6(global_values))]
                           
    return value_counter_image

def image_analysis(file):

    axI_Scene_Linear = file

    axI_Scene_Linear_Line_r = axI_Scene_Linear[:,:,0].reshape(-1, 1) * 100
    axI_Scene_Linear_Line_g = axI_Scene_Linear[:,:,1].reshape(-1, 1) * 100
    axI_Scene_Linear_Line_b = axI_Scene_Linear[:,:,2].reshape(-1, 1) * 100

    value_counter_image_r = split_array_to_stops(axI_Scene_Linear_Line_r[:,0])
    value_counter_image_g = split_array_to_stops(axI_Scene_Linear_Line_g[:,0])
    value_counter_image_b = split_array_to_stops(axI_Scene_Linear_Line_b[:,0])

    return np.dstack((value_counter_image_r,value_counter_image_g,value_counter_image_b))


In [ ]:
#@njit
def boolean_index_numba(array):
    index = (array[:, 0] >= 0) & (array[:,  0] <  1)
    return array[index]

In [ ]:
## Log C4 Encoding

import os
from fnmatch import fnmatch

root = '/Volumes/T7_Shield/XX_WATCHFOLDER_SSL'
pattern = "*.tiff"

print(root)

for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, pattern):
            path = os.path.join(path)
            print("hallo")
            new_path = path.replace('XX_WATCHFOLDER_SSL',"01_ANALYSIS")
            new_path_csv = new_path + "/CSV/"
            new_name = os.path.join(new_path,"SLL_Analysis_" + name)
            new_name = new_name.replace('.tiff',".png")
            new_name_csv = os.path.join(new_path_csv,"SLL_Analysis_" + name)
            new_csv = new_name_csv.replace(".tiff",".csv")
            print(new_name)
            print(new_csv)
            if not os.path.exists(new_path):
                os.makedirs(new_path)
            if not os.path.exists(new_path_csv):
                os.makedirs(new_path_csv)

            with open(new_csv, 'w', newline='') as file:
                writer = csv.writer(file)

                figR, (axI, axV) = plt.subplots(2, 1, figsize=(16, 16), constrained_layout=True, gridspec_kw={'height_ratios': [2, 1]})

                image_rgb = c.read_image(os.path.join(path,name))


                # axI_Scene_Linear = c.models.log_decoding_SLog3(image_rgb)
                # axI_Scene_Linear = c.models.log_decoding_ALEXALogC(image_rgb)
                # axI_Scene_Linear = c.models.log_decoding_ALEXALogC(c.LUT3D.apply(Inverse_LUT_ARRI_4K_PQ_200,image_rgb))
                # axI_Scene_Linear = Decoding_LogC4(image_rgb)
                axI_Display_Linear = c.models.eotf_ST2084(image_rgb)

                # axI_Scene_Linear_Line_r = axI_Scene_Linear[:,:,0].reshape(-1, 1)
                # axI_Scene_Linear_Line_g = axI_Scene_Linear[:,:,1].reshape(-1, 1)
                # axI_Scene_Linear_Line_b = axI_Scene_Linear[:,:,2].reshape(-1, 1)

                axI_Display_Linear_Line_r = axI_Display_Linear[:,:,0].reshape(-1, 1)/203
                axI_Display_Linear_Line_g = axI_Display_Linear[:,:,1].reshape(-1, 1)/203
                axI_Display_Linear_Line_b = axI_Display_Linear[:,:,2].reshape(-1, 1)/203
                axI_Display_Linear_Line_Luma = luma2020(image_rgb).reshape(-1, 1)/203

                value_counter_image_r = np.zeros((34))
                value_counter_image_g = np.zeros((34))
                value_counter_image_b = np.zeros((34))
                value_counter_image_luma = np.zeros((34))

                axI.imshow(c.LUT3D.apply(LUT_NBCU,image_rgb))
                # axI.imshow(c.LUT3D.apply(LUT_ARRI_C3_to_709_D65,image_rgb))
                # axI.imshow(c.LUT3D.apply(LUT_NBCU,image_rgb))
                # axI.imshow(c.LUT3D.apply(LUT_ARRI_C4_to_709_D65,image_rgb))

                axI.text(5,len(image_rgb[:,0,0])+(len(image_rgb[:,0,0])/20),f"HDR-SDR-Transform for SDR Display with 5-NBCU_PQ2SDR_DL_v1.cube", color='black', fontsize=10, fontstyle='normal')
                # axI.text(5,len(image_rgb[:,0,0])+(len(image_rgb[:,0,0]/20)),f"HDR-SDR-Transform for SDR Display with ARRI_LogC2Video_709_davinci3d_33.cube", color='black', fontsize=10, fontstyle='normal')


                axV.set_ylabel('Amount of Code Values Used [CV]')

                #x1 = np.linspace(0.000000000000001,1,1000000)

                axV.vlines(0.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(1.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(2.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(3.5,0,10000000,color='black', linestyle="solid", alpha=0.5,linewidth=2)
                axV.vlines(4.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axV.vlines(5.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axV.vlines(6.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axV.vlines(7.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axV.vlines(8.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axV.vlines(9.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axV.vlines(10.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
                axV.vlines(11.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(12.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(13.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(14.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(15.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(16.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(17.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)



                axV.vlines(1,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(2,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(3,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(4,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(5,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(6,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(7,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(8,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(9,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(10,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(11,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(12,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(13,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(14,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(15,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(16,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(17,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                
                value_counter_image_r = split_array_to_stops(axI_Display_Linear_Line_r[:,0])
                value_counter_image_g = split_array_to_stops(axI_Display_Linear_Line_g[:,0])
                value_counter_image_b = split_array_to_stops(axI_Display_Linear_Line_b[:,0])
                value_counter_image_luma = split_array_to_stops(axI_Display_Linear_Line_Luma[:,0])

                bars = ('A', 'B', 'C', 'D', 'E',
                    'F','G', 'H', 'I', 'J', 
                    'K','L','M', 'N', 'O', 
                    'P', 'Q','R','A', 'B', 
                    'C', 'D', 'E','F','G', 
                    'H', 'I', 'J', 'K','L',
                    'M', 'N', 'O', 'P')

                y_pos = np.arange(len(bars))

                axV.bar(y_pos/2-0.05+0.15, value_counter_image_luma, color="black", width=0.1, alpha=0.3, label="Display Linear Light Luma")
                axV.bar(y_pos/2+0.05+0.15, value_counter_image_r, color="red", width=0.1, alpha=0.3, label="Display Linear Light Red Channel")
                axV.bar(y_pos/2+0.15+0.15, value_counter_image_g, color="green", width=0.1, alpha=0.3, label="Display Linear Light Green Channel")
                axV.bar(y_pos/2+0.25+0.15, value_counter_image_b, color="blue", width=0.1, alpha=0.3, label="Display Linear Light Blue Channel")

                y_pos_between = [np.linspace(-10,7,18)]
                y_pos_neu = np.linspace(0,17,18)

                axV.set_xticks(y_pos_neu+0.5, y_pos_between)

                axV.set_xlim(-0.5,18.5)
                axV.set_yscale('log')
                axV.set_ylim(1,10000000)

                axV.legend()

                figR.savefig(new_name)
                writer.writerow([os.path.join("SLL_Analysis_" + name)])
                writer.writerow(value_counter_image_r)
                writer.writerow(value_counter_image_g)
                writer.writerow(value_counter_image_b)
                writer.writerow(["------------------------"])

# Working Analysis

In [ ]:
## Log C4 Encoding

import os
from fnmatch import fnmatch
from matplotlib.patches import Rectangle

root = '/Volumes/T7_Shield/XX_WATCHFOLDER_SSL'
pattern = "*.tif"

print(root)

for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, pattern):
            path = os.path.join(path)
            print("hallo")
            new_path = path.replace('XX_WATCHFOLDER_SSL',"01_ANALYSIS")
            new_path_csv = new_path + "/CSV/"
            new_name = os.path.join(new_path,"SLL_Analysis_" + name)
            new_name = new_name.replace('.tif',".png")
            new_name_csv = os.path.join(new_path_csv,"SLL_Analysis_" + name)
            new_csv = new_name_csv.replace(".tif",".csv")
            print(new_name)
            print(new_csv)
            if not os.path.exists(new_path):
                os.makedirs(new_path)
            if not os.path.exists(new_path_csv):
                os.makedirs(new_path_csv)

            with open(new_csv, 'w', newline='') as file:
                writer = csv.writer(file)

                figR, (axI, axL, axV) = plt.subplots(3, 1, figsize=(16, 20), constrained_layout=True, gridspec_kw={'height_ratios': [2, 1, 1]})

                image_rgb = c.read_image(os.path.join(path,name))


                # axI_Scene_Linear = c.models.log_decoding_SLog3(image_rgb)
                # axI_Scene_Linear = c.models.log_decoding_ALEXALogC(image_rgb)
                # axI_Scene_Linear = c.models.log_decoding_ALEXALogC(c.LUT3D.apply(Inverse_LUT_ARRI_4K_PQ_200,image_rgb))
                # axI_Scene_Linear = Decoding_LogC4(image_rgb)
                axI_Display_Linear = c.models.eotf_ST2084(image_rgb)

                # axI_Scene_Linear_Line_r = axI_Scene_Linear[:,:,0].reshape(-1, 1)
                # axI_Scene_Linear_Line_g = axI_Scene_Linear[:,:,1].reshape(-1, 1)
                # axI_Scene_Linear_Line_b = axI_Scene_Linear[:,:,2].reshape(-1, 1)

                axI_Display_Linear_Line_r = axI_Display_Linear[:,:,0].reshape(-1, 1)
                axI_Display_Linear_Line_g = axI_Display_Linear[:,:,1].reshape(-1, 1)
                axI_Display_Linear_Line_b = axI_Display_Linear[:,:,2].reshape(-1, 1)
                axI_Display_Linear_Line_Luma = luma2020(axI_Display_Linear).reshape(-1, 1)

                value_counter_image_r = np.zeros((36))
                value_counter_image_g = np.zeros((36))
                value_counter_image_b = np.zeros((36))
                value_counter_image_luma = np.zeros((36))

                axI.imshow(c.LUT3D.apply(LUT_NBCU,image_rgb))
                # axI.imshow(c.LUT3D.apply(LUT_ARRI_C3_to_709_D65,image_rgb))
                # axI.imshow(c.LUT3D.apply(LUT_NBCU,image_rgb))
                # axI.imshow(c.LUT3D.apply(LUT_ARRI_C4_to_709_D65,image_rgb))

                axI.text(5,len(image_rgb[:,0,0])+(len(image_rgb[:,0,0])/20),f"HDR-SDR-Transform for SDR Display with 5-NBCU_PQ2SDR_DL_v1.cube", color='black', fontsize=10, fontstyle='normal')
                # axI.text(5,len(image_rgb[:,0,0])+(len(image_rgb[:,0,0]/20)),f"HDR-SDR-Transform for SDR Display with ARRI_LogC2Video_709_davinci3d_33.cube", color='black', fontsize=10, fontstyle='normal')


                axL.set_ylabel('Amount of Code Values Used [CV]')
                axL.set_xlabel('Display Linear Light [cd/m2]')

                #x1 = np.linspace(0.000000000000001,1,1000000)

                axL.vlines(0.5+2/3,0,10000000,color='gray', linestyle="solid", alpha=1,linewidth=2)
                axL.vlines(1.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axL.vlines(2.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axL.vlines(3.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axL.vlines(4.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axL.vlines(5.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axL.vlines(6.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axL.vlines(7.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axL.vlines(8.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axL.vlines(9.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axL.vlines(10.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axL.vlines(11.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axL.vlines(12.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axL.vlines(13.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axL.vlines(14.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axL.vlines(15.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
                axL.vlines(16.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
                axL.vlines(17.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axL.vlines(18.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axL.vlines(18.5+0.25,0,10000000,color='black', linestyle="solid", alpha=1, linewidth=2)



                axL.vlines(2,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axL.vlines(3,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axL.vlines(4,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axL.vlines(5,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axL.vlines(6,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axL.vlines(7,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axL.vlines(8,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axL.vlines(9,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axL.vlines(10,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axL.vlines(11,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axL.vlines(12,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axL.vlines(13,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axL.vlines(14,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axL.vlines(15,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axL.vlines(16,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axL.vlines(17,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axL.vlines(18,0,10000000,color='gray', linestyle="dashed", alpha=0.4)

                value_counter_image_luma = split_array_to_stops(axI_Display_Linear_Line_Luma[:,0])

                bars = ('A', 'B', 'C', 'D', 'E',
                    'F','G', 'H', 'I', 'J', 
                    'K','L','M', 'N', 'O', 
                    'P', 'Q','R','A', 'B', 
                    'C', 'D', 'E','F','G', 
                    'H', 'I', 'J', 'K','L',
                    'M', 'N', 'O', 'P', 'Q' , 'R')

                y_pos = np.arange(len(bars))

                axL.bar(y_pos[1:35]/2 + 0.15 + (0.1) + 1, value_counter_image_luma[1:35], color="black", width=0.3, alpha=0.3, label="Display Linear Light Luma")
                axL.bar(y_pos[0:1]/2 + 0.15 + (0.1) + 1 + 0.1, value_counter_image_luma[0:1], color="black", width=0.1, alpha=0.3)
                axL.bar(y_pos[35:36]/2 + 0.15 + (0.1) + 1 - 0.125, value_counter_image_luma[35:36], color="black", width=0.1, alpha=0.3)

                print(value_counter_image_luma)

                y_pos_between = np.round([200 * 2 ** i for i in range(-15,3)],3)
                y_pos_neu = np.linspace(0,17,18)

                axL.set_xticks(y_pos_neu+1.5, y_pos_between)

                axL.set_xlim(0.5,19.5)
                axL.set_yscale('log')
                axL.set_ylim(1,10000000)


                axV.set_ylabel('Amount of Code Values Used [CV]')
                axV.set_xlabel('Display Linear Light [cd/m2]')

                #x1 = np.linspace(0.000000000000001,1,1000000)

                axV.vlines(0.5+2/3,0,10000000,color='gray', linestyle="solid", alpha=1,linewidth=2)
                axV.vlines(1.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(2.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(3.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(4.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(5.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(6.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(7.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(8.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(9.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(10.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axV.vlines(11.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axV.vlines(12.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axV.vlines(13.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axV.vlines(14.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axV.vlines(15.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
                axV.vlines(16.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
                axV.vlines(17.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(18.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(18.5+0.25,0,10000000,color='black', linestyle="solid", alpha=1, linewidth=2)




                axV.vlines(2,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(3,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(4,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(5,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(6,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(7,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(8,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(9,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(10,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(11,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(12,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(13,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(14,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(15,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(16,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(17,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(18,0,10000000,color='gray', linestyle="dashed", alpha=0.4)

                
                value_counter_image_r = split_array_to_stops(axI_Display_Linear_Line_r[:,0])
                value_counter_image_g = split_array_to_stops(axI_Display_Linear_Line_g[:,0])
                value_counter_image_b = split_array_to_stops(axI_Display_Linear_Line_b[:,0])



                axV.bar(y_pos[1:35]/2+0.0+0.15+1, value_counter_image_r[1:35], color="red", width=0.1, alpha=0.3, label="Display Linear Light Red")
                axV.bar(y_pos[1:35]/2+0.1+0.15+1, value_counter_image_g[1:35], color="green", width=0.1, alpha=0.3, label="Display Linear Light Green")
                axV.bar(y_pos[1:35]/2+0.2+0.15+1, value_counter_image_b[1:35], color="blue", width=0.1, alpha=0.3, label="Display Linear Light Blue")

                axV.bar(y_pos[0:1]/2+0.0+0.3+1, value_counter_image_r[0:1], color="red", width=0.05, alpha=0.3)
                axV.bar(y_pos[0:1]/2+0.05+0.3+1, value_counter_image_g[0:1], color="green", width=0.05, alpha=0.3)
                axV.bar(y_pos[0:1]/2+0.1+0.3+1, value_counter_image_b[0:1], color="blue", width=0.05, alpha=0.3)

                axV.bar(y_pos[35:36]/2+0.0+1+0.075, value_counter_image_r[35:36], color="red", width=0.05, alpha=0.3)
                axV.bar(y_pos[35:36]/2+0.05+1+0.075, value_counter_image_g[35:36], color="green", width=0.05, alpha=0.3)
                axV.bar(y_pos[35:36]/2+0.1+1+0.075, value_counter_image_b[35:36], color="blue", width=0.05, alpha=0.3)

                axV.legend(facecolor='white', framealpha=1)
                axL.legend(facecolor='white', framealpha=1)

                axV.add_patch(Rectangle((0.5+2/3, 0), -100, 10000000,facecolor = 'black',fill=True,alpha=0.2))
                axL.add_patch(Rectangle((0.5+2/3, 0), -100, 10000000,facecolor = 'black',fill=True,alpha=0.2))
                axV.add_patch(Rectangle((18.75, 0), 100, 10000000,facecolor = 'black',fill=True,alpha=0.2))
                axL.add_patch(Rectangle((18.75, 0), 100, 10000000,facecolor = 'black',fill=True,alpha=0.2))

                axL.set_title("Code Value Distribution - Display Linear Light // Luma")
                axV.set_title("Code Value Distribution - Display Linear Light // RGB")

                axV.set_xticks(y_pos_neu+0.5, y_pos_between)

                axV.set_xlim(0.5,19.5)
                axV.set_yscale('log')
                axV.set_ylim(1,10000000)

                figR.savefig(new_name)
                writer.writerow([os.path.join("SLL_Analysis_" + name)])
                writer.writerow(value_counter_image_r)
                writer.writerow(value_counter_image_g)
                writer.writerow(value_counter_image_b)
                writer.writerow(value_counter_image_luma)
                writer.writerow(["------------------------"])

In [ ]:
test0202 = c.read_image("/Volumes/T7_Shield/XX_WATCHFOLDER_SSL/11.1_TRANSFORMED_PQ_FR_UHD/ARRI_Encounters/1_B002C026_220414_B09C/TRANSFORMED_PQ_BT2020_FR_UHD_B002C026_220414_B09C.00000000.tif")

In [ ]:
c.models.eotf_ST2084(0.73)

In [ ]:
luma_array = luma2020(test0202)

In [ ]:
filtered_array = luma_array[luma_array > 0.7]
print(len(filtered_array))

# Code for Percentage

In [ ]:
## Log C4 Encoding

import os
from fnmatch import fnmatch
from matplotlib.patches import Rectangle
import re

root = '/Volumes/T7_Shield/XX_WATCHFOLDER_SSL'
pattern = "*.tif"

print(root)

for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, pattern):
            path = os.path.join(path)
            print("hallo")
            new_path = path.replace('XX_WATCHFOLDER_SSL',"01_ANALYSIS")
            new_path_csv = new_path + "/CSV/"
            new_name = os.path.join(new_path,"SLL_Analysis_" + name)
            new_name = new_name.replace('.tif',".png")
            new_name_csv = os.path.join(new_path_csv,"SLL_Analysis_" + name)
            new_csv = new_name_csv.replace(".tif",".csv")
            print(new_name)
            print(new_csv)
            if not os.path.exists(new_path):
                os.makedirs(new_path)
            if not os.path.exists(new_path_csv):
                os.makedirs(new_path_csv)

            with open(new_csv, 'w', newline='') as file:
                writer = csv.writer(file)

                figR, (axI, axL, axV) = plt.subplots(3, 1, figsize=(16, 20), constrained_layout=True, gridspec_kw={'height_ratios': [2, 1, 1]})

                image_rgb = c.read_image(os.path.join(path,name))
                input_string = os.path.join(path,name)

                pattern1 = re.compile(r'HLG')
                pattern2 = re.compile(r'PQ')

                # Check if the pattern is present in the input string
                if pattern1.search(input_string):
                    # Do something if the pattern is found
                    print("HLG!!!")
                    axI_Display_Linear = c.models.eotf_HLG_BT2100(image_rgb,0,1000)
                    axI.imshow(c.LUT3D.apply(LUT_NBCU_HLG,image_rgb))
                    axI.text(5,len(image_rgb[:,0,0])+(len(image_rgb[:,0,0])/20),f"HDR-SDR-Transform for SDR Display with 3-NBCU_HLG2SDR_DL_v1.1.cube", color='black', fontsize=10, fontstyle='normal')
                
                    # Add your code here to perform the desired action
                if pattern2.search(input_string):
                    print("PQ!!!")
                    axI_Display_Linear = c.models.eotf_ST2084(image_rgb)
                    axI.imshow(c.LUT3D.apply(LUT_NBCU,image_rgb))
                    axI.text(5,len(image_rgb[:,0,0])+(len(image_rgb[:,0,0])/20),f"HDR-SDR-Transform for SDR Display with 5-NBCU_PQ2SDR_DL_v1.cube", color='black', fontsize=10, fontstyle='normal')
                

                # axI_Scene_Linear = c.models.log_decoding_SLog3(image_rgb)
                # axI_Scene_Linear = c.models.log_decoding_ALEXALogC(image_rgb)
                # axI_Scene_Linear = c.models.log_decoding_ALEXALogC(c.LUT3D.apply(Inverse_LUT_ARRI_4K_PQ_200,image_rgb))
                # axI_Scene_Linear = Decoding_LogC4(image_rgb)
                

                # axI_Scene_Linear_Line_r = axI_Scene_Linear[:,:,0].reshape(-1, 1)
                # axI_Scene_Linear_Line_g = axI_Scene_Linear[:,:,1].reshape(-1, 1)
                # axI_Scene_Linear_Line_b = axI_Scene_Linear[:,:,2].reshape(-1, 1)

                axI_Display_Linear_Line_r = axI_Display_Linear[:,:,0].reshape(-1, 1)
                axI_Display_Linear_Line_g = axI_Display_Linear[:,:,1].reshape(-1, 1)
                axI_Display_Linear_Line_b = axI_Display_Linear[:,:,2].reshape(-1, 1)
                axI_Display_Linear_Line_Luma = luma2020(axI_Display_Linear).reshape(-1, 1)

                value_counter_image_r = np.zeros((36))
                value_counter_image_g = np.zeros((36))
                value_counter_image_b = np.zeros((36))
                value_counter_image_luma = np.zeros((36))
                value_counter_image_hlg_FR = np.zeros((36))
                value_counter_image_pq_FR = np.zeros((36))

                
                # axI.imshow(c.LUT3D.apply(LUT_ARRI_C3_to_709_D65,image_rgb))
                # axI.imshow(c.LUT3D.apply(LUT_NBCU,image_rgb))
                # axI.imshow(c.LUT3D.apply(LUT_ARRI_C4_to_709_D65,image_rgb))

                # axI.text(5,len(image_rgb[:,0,0])+(len(image_rgb[:,0,0]/20)),f"HDR-SDR-Transform for SDR Display with ARRI_LogC2Video_709_davinci3d_33.cube", color='black', fontsize=10, fontstyle='normal')


                axL.set_ylabel('Amount of Code Values Used [%]')
                axL.set_xlabel('Display Linear Light [cd/m2]')

                #x1 = np.linspace(0.000000000000001,1,1000000)

                axL.vlines(0.5+2/3,0,10000000,color='black', linestyle="solid", alpha=1,linewidth=2)
                axL.vlines(1.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axL.vlines(2.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axL.vlines(3.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axL.vlines(4.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axL.vlines(5.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axL.vlines(6.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axL.vlines(7.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axL.vlines(8.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axL.vlines(9.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axL.vlines(10.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axL.vlines(11.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axL.vlines(12.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axL.vlines(13.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axL.vlines(14.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axL.vlines(15.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
                axL.vlines(16.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
                axL.vlines(17.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axL.vlines(18.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axL.vlines(18.5+0.25,0,10000000,color='black', linestyle="solid", alpha=1, linewidth=2)



                axL.vlines(2,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axL.vlines(3,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axL.vlines(4,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axL.vlines(5,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axL.vlines(6,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axL.vlines(7,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axL.vlines(8,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axL.vlines(9,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axL.vlines(10,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axL.vlines(11,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axL.vlines(12,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axL.vlines(13,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axL.vlines(14,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axL.vlines(15,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axL.vlines(16,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axL.vlines(17,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axL.vlines(18,0,10000000,color='gray', linestyle="dashed", alpha=0.4)


                

                value_counter_image_luma = split_array_to_stops(axI_Display_Linear_Line_Luma[:,0])
                value_counter_image_hlg_FR = split_array_to_stops(c.models.eotf_HLG_BT2100(legal2full(np.linspace(64,940,877)),0,1000))
                value_counter_image_pq_FR = split_array_to_stops(c.models.eotf_ST2084(legal2full(np.linspace(64,940,877))))

                print(value_counter_image_luma)

                bars = ('A', 'B', 'C', 'D', 'E',
                    'F','G', 'H', 'I', 'J', 
                    'K','L','M', 'N', 'O', 
                    'P', 'Q','R','A', 'B', 
                    'C', 'D', 'E','F','G', 
                    'H', 'I', 'J', 'K','L',
                    'M', 'N', 'O', 'P', 'Q' , 'R')

                y_pos = np.arange(len(bars))

                axL.step(y_pos/2 + 1.25, np.divide(value_counter_image_hlg_FR,sum(value_counter_image_hlg_FR)), 'k',where="mid",linewidth=1,color="darkturquoise", label="HLG EOTF 10 bit Narrow Range",alpha=0.7)
                axL.step(y_pos/2 + 1.25, np.divide(value_counter_image_pq_FR,sum(value_counter_image_pq_FR)), 'k',where="mid",linewidth=1,color="darkgreen", label="PQ EOTF 10 bit Narrow Range",alpha=0.7)

                axL.bar(y_pos[1:35]/2 + 0.15 + (0.1) + 1, np.divide(value_counter_image_luma[1:35],sum(value_counter_image_luma)), color="black", width=0.3, alpha=0.3, label="Image Display Linear Light Luma")
                axL.bar(y_pos[0:1]/2 + 0.15 + (0.1) + 1 + 0.1, np.divide(value_counter_image_luma[0:1],sum(value_counter_image_luma)), color="black", width=0.1, alpha=0.3)
                axL.bar(y_pos[35:36]/2 + 0.15 + (0.1) + 1 - 0.125, np.divide(value_counter_image_luma[35:36],sum(value_counter_image_luma)), color="black", width=0.1, alpha=0.3)
                
                #axL.bar(y_pos/2 + 1.25, np.divide(value_counter_image_hlg_FR,sum(value_counter_image_hlg_FR)), color="black", width=0.5, alpha=0.3, label="HLG EOTF 10 bit Narrow Range")
                #axL.bar(y_pos/2 + 1.25, np.divide(value_counter_image_hlg_FR,sum(value_counter_image_pq_FR)), color="black", width=0.5, alpha=0.3, label="PQ EOTF 10 bit Narrow Range")
                
                y_pos_between = np.round([200 * 2 ** i for i in range(-15,3)],3)
                y_pos_neu = np.linspace(0,17,18)

                axL.set_xticks(y_pos_neu+1.5, y_pos_between)

                axL.set_xlim(0.5,19.5)
                axL.set_yscale("log")
                axL.set_ylim(0.0001,1)
                axL.set_yticks([0.0001,0.001,0.01,0.1,1],[0.01,0.1,1,10,100])


                axV.set_ylabel('Amount of Code Values Used [%]')
                axV.set_xlabel('Display Linear Light [cd/m2]')

                #x1 = np.linspace(0.000000000000001,1,1000000)

                axV.vlines(0.5+2/3,0,10000000,color='black', linestyle="solid", alpha=1,linewidth=2)
                axV.vlines(1.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(2.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(3.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(4.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(5.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(6.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(7.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(8.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(9.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(10.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axV.vlines(11.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axV.vlines(12.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axV.vlines(13.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axV.vlines(14.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axV.vlines(15.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
                axV.vlines(16.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
                axV.vlines(17.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(18.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(18.5+0.25,0,10000000,color='black', linestyle="solid", alpha=1, linewidth=2)


                axV.vlines(2,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(3,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(4,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(5,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(6,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(7,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(8,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(9,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(10,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(11,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(12,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(13,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(14,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(15,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(16,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(17,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(18,0,10000000,color='gray', linestyle="dashed", alpha=0.4)

                
                value_counter_image_r = split_array_to_stops(axI_Display_Linear_Line_r[:,0])
                value_counter_image_g = split_array_to_stops(axI_Display_Linear_Line_g[:,0])
                value_counter_image_b = split_array_to_stops(axI_Display_Linear_Line_b[:,0])

                axV.step(y_pos/2 + 1.25, np.divide(value_counter_image_hlg_FR,sum(value_counter_image_hlg_FR)), 'k',where="mid",linewidth=1,color="darkturquoise", label="HLG EOTF 10 bit Narrow Range",alpha=0.7)
                axV.step(y_pos/2 + 1.25, np.divide(value_counter_image_pq_FR,sum(value_counter_image_pq_FR)), 'k',where="mid",linewidth=1,color="darkgreen", label="PQ EOTF 10 bit Narrow Range",alpha=0.7)


                axV.bar(y_pos[1:35]/2+0.0+0.15+1, np.divide(value_counter_image_r[1:35],sum(value_counter_image_r)), color="red", width=0.1, alpha=0.3, label="Image Display Linear Light Red")
                axV.bar(y_pos[1:35]/2+0.1+0.15+1, np.divide(value_counter_image_g[1:35],sum(value_counter_image_g)), color="green", width=0.1, alpha=0.3, label="Image Display Linear Light Green")
                axV.bar(y_pos[1:35]/2+0.2+0.15+1, np.divide(value_counter_image_b[1:35],sum(value_counter_image_b)), color="blue", width=0.1, alpha=0.3, label="Image Display Linear Light Blue")

                axV.bar(y_pos[0:1]/2+0.0+0.3+1+0.05, np.divide(value_counter_image_r[0:1],sum(value_counter_image_r)), color="red", width=0.05, alpha=0.3)
                axV.bar(y_pos[0:1]/2+0.05+0.3+1+0.05, np.divide(value_counter_image_g[0:1],sum(value_counter_image_g)), color="green", width=0.05, alpha=0.3)
                axV.bar(y_pos[0:1]/2+0.1+0.3+1+0.05, np.divide(value_counter_image_b[0:1],sum(value_counter_image_b)), color="blue", width=0.05, alpha=0.3)

                axV.bar(y_pos[35:36]/2+0.0+1+0.075, np.divide(value_counter_image_r[35:36],sum(value_counter_image_r)), color="red", width=0.05, alpha=0.3)
                axV.bar(y_pos[35:36]/2+0.05+1+0.075, np.divide(value_counter_image_g[35:36],sum(value_counter_image_g)), color="green", width=0.05, alpha=0.3)
                axV.bar(y_pos[35:36]/2+0.1+1+0.075, np.divide(value_counter_image_b[35:36],sum(value_counter_image_b)), color="blue", width=0.05, alpha=0.3)

                axV.legend(facecolor='white', framealpha=1)
                axL.legend(facecolor='white', framealpha=1)

                axV.add_patch(Rectangle((0.5+2/3, 0), -100, 10000000,facecolor = 'lightgray',fill=True,alpha=1))
                axL.add_patch(Rectangle((0.5+2/3, 0), -100, 10000000,facecolor = 'lightgray',fill=True,alpha=1))
                axV.add_patch(Rectangle((18.75, 0), 100, 10000000,facecolor = 'lightgray',fill=True,alpha=1))
                axL.add_patch(Rectangle((18.75, 0), 100, 10000000,facecolor = 'lightgray',fill=True,alpha=1))

                axL.set_title("Code Value Distribution - Display Linear Light // Luma")
                axV.set_title("Code Value Distribution - Display Linear Light // RGB")

                axV.set_xticks(y_pos_neu+1.5, y_pos_between)
                axV.text(0.88,0.01,"0.005 cd/m2 Limit BT.2100 Ref. Display",rotation=90,horizontalalignment="center",verticalalignment="center",fontsize=14)
                axL.text(0.88,0.01,"0.005 cd/m2 Limit BT.2100 Ref. Display",rotation=90,horizontalalignment="center",verticalalignment="center",fontsize=14)
                axV.text(19.16,0.01,"1000 cd/m2 Limit BT.2100 Ref. Display",rotation=90,horizontalalignment="center",verticalalignment="center",fontsize=14)
                axL.text(19.16,0.01,"1000 cd/m2 Limit BT.2100 Ref. Display",rotation=90,horizontalalignment="center",verticalalignment="center",fontsize=14)

                axV.set_xlim(0.5,19.5)
                axV.set_yscale("log")
                axV.set_ylim(0.0001,1)
                axV.set_yticks([0.0001,0.001,0.01,0.1,1],[0.01,0.1,1,10,100])

                figR.savefig(new_name)
                writer.writerow([os.path.join("SLL_Analysis_" + name)])
                writer.writerow(value_counter_image_r)
                writer.writerow(value_counter_image_g)
                writer.writerow(value_counter_image_b)
                writer.writerow(value_counter_image_luma)
                writer.writerow(["------------------------"])

In [ ]:
import numpy as np

# Create a 3D array with dimensions 1024x1024x1024
image_array = np.zeros((1024, 1024, 3), dtype=np.uint16)

# Assign pixel values from 0 to 1023 in the first dimension (left to right)
for i in range(1024):
    image_array[i, :, :] = i

# All rows are the same in the second dimension
image_array[:, 0, :] = 0

# All layers are the same in the third dimension
image_array[:, :, 0] = 0

# Print the resulting 3D array
print(image_array)

c.write_image(image_array*(2**6),"/Users/justus/Desktop/Test_array.tiff",bit_depth="uint16")

In [ ]:
## Log C4 Encoding

import os
from fnmatch import fnmatch

root = '/Volumes/T7_Shield/XX_WATCHFOLDER_SSL'
pattern = "*.tiff"

print(root)

for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, pattern):
            path = os.path.join(path)
            print("hallo")
            new_path = path.replace('XX_WATCHFOLDER_SSL',"01_ANALYSIS")
            new_path_csv = new_path + "/CSV/"
            new_name = os.path.join(new_path,"SLL_Analysis_" + name)
            new_name = new_name.replace('.tiff',".png")
            new_name_csv = os.path.join(new_path_csv,"SLL_Analysis_" + name)
            new_csv = new_name_csv.replace(".tiff",".csv")
            print(new_name)
            print(new_csv)
            if not os.path.exists(new_path):
                os.makedirs(new_path)
            if not os.path.exists(new_path_csv):
                os.makedirs(new_path_csv)

            with open(new_csv, 'w', newline='') as file:
                writer = csv.writer(file)

                figR, (axI, axV) = plt.subplots(2, 1, figsize=(16, 16), constrained_layout=True, gridspec_kw={'height_ratios': [2, 1]})

                image_rgb = c.read_image(os.path.join(path,name))


                # axI_Scene_Linear = c.models.log_decoding_SLog3(image_rgb)
                # axI_Scene_Linear = c.models.log_decoding_ALEXALogC(image_rgb)
                # axI_Scene_Linear = c.models.log_decoding_ALEXALogC(c.LUT3D.apply(Inverse_LUT_ARRI_4K_PQ_200,image_rgb))
                # axI_Scene_Linear = Decoding_LogC4(image_rgb)
                axI_Display_Linear = c.models.eotf_ST2084(image_rgb)

                # axI_Scene_Linear_Line_r = axI_Scene_Linear[:,:,0].reshape(-1, 1)
                # axI_Scene_Linear_Line_g = axI_Scene_Linear[:,:,1].reshape(-1, 1)
                # axI_Scene_Linear_Line_b = axI_Scene_Linear[:,:,2].reshape(-1, 1)

                axI_Display_Linear_Line_r = axI_Display_Linear[:,:,0].reshape(-1, 1)/203
                axI_Display_Linear_Line_g = axI_Display_Linear[:,:,1].reshape(-1, 1)/203
                axI_Display_Linear_Line_b = axI_Display_Linear[:,:,2].reshape(-1, 1)/203
                axI_Display_Linear_Line_Luma = luma2020(image_rgb).reshape(-1, 1)/203

                value_counter_image_r = np.zeros((34))
                value_counter_image_g = np.zeros((34))
                value_counter_image_b = np.zeros((34))
                value_counter_image_luma = np.zeros((34))

                axI.imshow(c.LUT3D.apply(LUT_NBCU,image_rgb))
                # axI.imshow(c.LUT3D.apply(LUT_ARRI_C3_to_709_D65,image_rgb))
                # axI.imshow(c.LUT3D.apply(LUT_NBCU,image_rgb))
                # axI.imshow(c.LUT3D.apply(LUT_ARRI_C4_to_709_D65,image_rgb))

                axI.text(5,len(image_rgb[:,0,0])+(len(image_rgb[:,0,0])/20),f"HDR-SDR-Transform for SDR Display with 5-NBCU_PQ2SDR_DL_v1.cube", color='black', fontsize=10, fontstyle='normal')
                # axI.text(5,len(image_rgb[:,0,0])+(len(image_rgb[:,0,0]/20)),f"HDR-SDR-Transform for SDR Display with ARRI_LogC2Video_709_davinci3d_33.cube", color='black', fontsize=10, fontstyle='normal')


                axV.set_ylabel('Amount of Code Values Used [CV]')

                #x1 = np.linspace(0.000000000000001,1,1000000)

                axV.vlines(0.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(1.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(2.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(3.5,0,10000000,color='black', linestyle="solid", alpha=0.5,linewidth=2)
                axV.vlines(4.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axV.vlines(5.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axV.vlines(6.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axV.vlines(7.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axV.vlines(8.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axV.vlines(9.5,0,10000000,color='black', linestyle="solid", alpha=0.4)
                axV.vlines(10.5,0,10000000,color='black', linestyle="solid", alpha=0.5, linewidth=2)
                axV.vlines(11.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(12.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)
                axV.vlines(13.5,0,10000000,color='gray', linestyle="solid", alpha=0.4)


                axV.vlines(1,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(2,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(3,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(4,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(5,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(6,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(7,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(8,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(9,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(10,0,10000000,color='black', linestyle="dashed", alpha=0.4)
                axV.vlines(11,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(12,0,10000000,color='gray', linestyle="dashed", alpha=0.4)
                axV.vlines(13,0,10000000,color='gray', linestyle="dashed", alpha=0.4)

                
                value_counter_image_r = split_array_to_stops(axI_Display_Linear_Line_r[:,0])
                value_counter_image_g = split_array_to_stops(axI_Display_Linear_Line_g[:,0])
                value_counter_image_b = split_array_to_stops(axI_Display_Linear_Line_b[:,0])
                value_counter_image_luma = split_array_to_stops(axI_Display_Linear_Line_Luma[:,0])

                bars = ('A', 'B', 'C', 'D', 'E',
                    'F','G', 'H', 'I', 'J', 
                    'K','L','M', 'N', 'O', 
                    'P', 'Q','R','A', 'B', 
                    'C', 'D', 'E','F','G', 
                    'H', 'I', 'J', 'K','L',
                    'M', 'N', 'O', 'P')

                y_pos = np.arange(len(bars))

                axV.bar(y_pos/2-0.05+0.15, value_counter_image_luma, color="black", width=0.1, alpha=0.3, label="Display Linear Light Luma")
                axV.bar(y_pos/2+0.05+0.15, value_counter_image_r, color="red", width=0.1, alpha=0.3, label="Display Linear Light Red Channel")
                axV.bar(y_pos/2+0.15+0.15, value_counter_image_g, color="green", width=0.1, alpha=0.3, label="Display Linear Light Green Channel")
                axV.bar(y_pos/2+0.25+0.15, value_counter_image_b, color="blue", width=0.1, alpha=0.3, label="Display Linear Light Blue Channel")

                y_pos_between = np.round([1 * 10 ** i for i in range(-3,4)],3)
                y_pos_neu = np.linspace(0,6,7)

                axV.set_xticks(y_pos_neu+0.5, y_pos_between)

                axV.set_xlim(-0.5,7.5)
                axV.set_yscale('log')
                axV.set_ylim(1,10000000)

                axV.legend()

                # figR.savefig(new_name)
                # writer.writerow([os.path.join("SLL_Analysis_" + name)])
                # writer.writerow(value_counter_image_r)
                # writer.writerow(value_counter_image_g)
                # writer.writerow(value_counter_image_b)
                # writer.writerow(["------------------------"])